In [ ]:
## import python modules
import numpy as np
import xarray as xr
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import glob
import pandas as pd

import cartopy
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.util import add_cyclic_point
import matplotlib
import matplotlib.cm as cm
from matplotlib.colors import BoundaryNorm
import datetime as dt

#from matplotlib import animation
from matplotlib.animation import FuncAnimation
# from IPython.display import HTML
# import matplotlib.animation as animation

# from matplotlib import rc
import matplotlib
#plt.rcParams['animation.ffmpeg_path'] = '/usr/local/bin/ffmpeg'
%matplotlib inline
from PIL import Image, ImageDraw
from matplotlib import animation
from mpl_toolkits.axes_grid1 import make_axes_locatable
import math
import matplotlib.image as mgimg
from collections import Counter
from global_land_mask import globe
import scipy.optimize as opt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# set paths to read buoyancy and precipitation data, The data is spaced 6 hours apart with a horizontal resolution of $\sim$ 0.25 deg 

In [ ]:
buoy_path ='/neelin2020/ERA-5_buoy/layer_thetae/era5_2layers_thetae_'
precip_path='/neelin2020/TRMM_3B42/TRMM.3hrly.3B42.'
number = []
offset = []
non_landfall = []
tropics = []


class hurricane reads year and extracts number of TCs, and related thermodynamic and precipitation fields from ERA5 data

In [ ]:
number = []
offset = []
non_landfall = []
landfall = []
tropics = []

class hurricance:
    
    def __init__(self,year,paths):
        self.year=year
        self.buoy_path=paths[0]
        self.precip_path=paths[1]
        #self.buoy_path=paths[2]
        self.offset=5 ## box size in deg. lat/lon around feature center
        offset.append(self.offset)
    def __extract_dates(self,date_array):
        dates=[]
        for n in range(date_array.shape[0]):
            yr,mo,dy,hr=np.int_(date_array[n])
            dates.append(dt.datetime(yr,mo,dy,hr))
        
        return dates    
    
    def extract_tracks(self):
        
        ### read Tempest Extremes Tracks for ERA-5
        fili='/neelin2020/TempestExtremes/TC/ERA5_TC_tracks_20200824.txt'
        
        f=open(fili,'r')
        lines = f.readlines()    # Read file and close
        f.close()

        i=0
        self.tc_tracks={}
        ctr=0

        while (i < (len(lines)-1) ):
            line = lines[i]
            i = i+1
            linesplit = line.strip().split('\t')
            if linesplit[0] == 'start' :
                tracklength = int(linesplit[1])
                trackarray = np.genfromtxt(lines[i:(i+tracklength)])
                dates=self.__extract_dates(np.int_(trackarray[:,-4:]))
                lon=trackarray[:,2]
                lat=trackarray[:,3]
                psl=trackarray[:,4]*1e-2 ### in hPa
                max_wind=trackarray[:,5]
                
        #         if(np.logical_and(dates[0].year>start_year, dates[-1].year<=end_year)): ## for start and end years
                if(dates[0].year==self.year & dates[-1].year==self.year):
                    self.tc_tracks[ctr]={}
                    self.tc_tracks[ctr]['lat']=lat
                    self.tc_tracks[ctr]['lon']=lon
                    self.tc_tracks[ctr]['dates']=dates
                    self.tc_tracks[ctr]['psl']=psl
                    self.tc_tracks[ctr]['maxwind']=max_wind
                    ctr+=1
        print('There are {:d} Tropical Cyclones in {:d}'.format(ctr,self.year))
        number.append(ctr)
    def read_data(self):
        
        '''
        This is the function that reads the buoyancy 
        fields from the ERA5 product.
        '''

        self.ds_comp={}
        
        for key in self.tc_tracks.keys():

            fil_era5=glob.glob(self.buoy_path+'2002_01_01.nc')[0]
            ds_buoy=xr.open_dataset(fil_era5)
            ds_buoy.close()
            
            dates=self.tc_tracks[key]['dates']
            
            nan_array=np.zeros((len(dates),ds_buoy.lat.size,ds_buoy.lon.size))
            nan_array[:]=np.nan    
                

            self.ds_comp[key] = xr.Dataset(
                data_vars=dict(
                    #crh=(["time","lat", "lon"], nan_array.copy()),
                    CAPE =(["time","lat", "lon"], nan_array.copy()),
                    Subsaturation=(["time","lat", "lon"], nan_array.copy()),
                    Buoyancy=(["time","lat", "lon"], nan_array.copy()),
                    EQBL=(["time","lat", "lon"], nan_array.copy()),
                    EQSLFT=(["time","lat", "lon"], nan_array.copy()),
                    EQULFT=(["time","lat", "lon"], nan_array.copy()),
                    precipitation=(["time","lat", "lon"], nan_array.copy()),
                ),
                coords=dict(
                    lon=(ds_buoy.lon),
                    lat=(ds_buoy.lat),
                    time=dates,
                ))
                
    #print(self.buoy_path+'2002_01_01.nc')     
                                    
            for i,j,k in zip(self.tc_tracks[key]['lat'],self.tc_tracks[key]['lon'],self.tc_tracks[key]['dates']):
                try:
                    fil_era5=glob.glob(self.buoy_path+dt.datetime.strftime(k,'%Y_%m_%d')+'.nc')[0]
                    fil_precip=glob.glob(self.precip_path+dt.datetime.strftime(k,'%Y%m*')+'.nc')[0]
                except:
                    print('Files not found')
                    break
                

                ### extract lat, lon ranges for each date ###
                ds_buoy=xr.open_dataset(fil_era5)
                ds_precip=xr.open_dataset(fil_precip)
                ds_precip=ds_precip.rename({'latitude':'lat',
                                           'longitude':'lon'})
                
                la=ds_buoy.lat
                lc=ds_buoy.lon

                loc_dict=dict(lat=la[(la>=i-self.offset) & (la<=i+self.offset)],
                       lon=lc[(lc>=j-self.offset) & (lc<=j+self.offset)])

                loc_dict_time=dict(time=k,
                                   lat=la[(la>=i-self.offset) & (la<=i+self.offset)],
                       lon=lc[(lc>=j-self.offset) & (lc<=j+self.offset)])

                
                if len(loc_dict['lat'])==0:
                    print('Latitude out of bounds')
                    break

                thetae_bl=ds_buoy.thetae_bl.sel(time=k,method='nearest').loc[loc_dict]
                #print(thetae_bl)
                thetae_lt=ds_buoy.thetae_lt.sel(time=k,method='nearest').loc[loc_dict]
                thetae_sat_lt=ds_buoy.thetae_sat_lt.sel(time=k,method='nearest').loc[loc_dict]
                #print(thetae_sat_lt)
#                 print([loc_dict])
                CAPE_new = ((thetae_bl-thetae_sat_lt)/thetae_sat_lt)*340
                SUBSAT_new = ((thetae_sat_lt-thetae_lt)/thetae_sat_lt)*340
                BUOY_new = (9.8/(340*3))*((.5*CAPE_new)-(.5*SUBSAT_new))
                #crh_inst=cwv/cwv_sat
                self.ds_comp[key]['CAPE'].loc[loc_dict_time]=CAPE_new
                
                self.ds_comp[key]['Subsaturation'].loc[loc_dict_time]=SUBSAT_new
                
                self.ds_comp[key]['Buoyancy'].loc[loc_dict_time]=BUOY_new
                
                self.ds_comp[key]['EQBL'].loc[loc_dict_time]=thetae_bl
                self.ds_comp[key]['EQSLFT'].loc[loc_dict_time]=thetae_sat_lt
                self.ds_comp[key]['EQULFT'].loc[loc_dict_time]=thetae_lt
                prc_inst=ds_precip.precip_trmm.sel(time=k,method='nearest').loc[loc_dict]
                self.ds_comp[key]['precipitation'].loc[loc_dict_time]=prc_inst
                

                
                ds_buoy.close()
                ds_precip.close()
            
            self.ds_comp[key].close()
            print(key, 'Max wind: {:.2f} m/s, Min psl: {:.2f} hPa'.format(max(self.tc_tracks[key]['maxwind']), min(self.tc_tracks[key]['psl'])))
            print(tcs.ds_comp[key].time.dt.strftime("%Y%m%d-%H").isel(time=0).item())
            long_array=tcs.tc_tracks[key]['lon'] #taking latitude and longitude arrays
            lati_array=tcs.tc_tracks[key]['lat']
            breaker = 0
            breakers = 0
            for lan in range(len(long_array)):
                if long_array[lan] >= 180: #since the landmask array takes negatives instead of 360
                    long_array[lan] = long_array[lan]-360
                if globe.is_land(lati_array[lan], long_array[lan]) == True: #if one value is on the landmask
                    
                    if breaker == 0:
                        breaker = breaker+1 #so it doesn't print landfall for every point
                        print("Landfalls at some point")
                        landfall.append(key) #append to landfall list
                if globe.is_land(lati_array[lan], long_array[lan]) == False: #for storms that never hit the landmask
                    breakers = breakers + 1
                if long_array[lan] < 0:
                    long_array[lan] = long_array[lan]+360
                if breakers == len(lati_array): #if every position is not over land
                    non_landfall.append(key)#append to nonlandfall list
            passer = 0 
            for lin in range(len(lati_array)):
                if (lati_array[lin] >= (-34.75+offset[0])) and (lati_array[lin] <= (34.75-offset[0])):
                    passer = passer + 1
                    #if the box around a storm passes 35 degree latitude or not
                if passer == len(lati_array): #if all positions are within the tropics
                    print("Stays in the Tropics")
                    tropics.append(key)
            #so I know what basin a storm belongs to (isn't perfect)
            if long_array[0] >= 261 and long_array[0] <= 359:
                print("Atlantic")
            elif long_array[0] >= 220 and long_array[0] < 261 and lati_array[0] >= 0:
                print("East Pacific")
            elif long_array[0] >= 180 and long_array[0] < 220 and lati_array[0] >= 0:
                print("Central Pacific")
            elif long_array[0] >= 100 and long_array[0] < 180 and lati_array[0] >= 0:
                print("West Pacific")
            elif long_array[0] >= 45 and long_array[0] < 100 and lati_array[0] >= 0:
                print("North Indian")
            elif long_array[0] >= 20  and long_array[0] < 90 and lati_array[0] < 0:
                print("Southwest Indian")
            elif long_array[0] >= 90  and long_array[0] < 160 and lati_array[0] < 0:
                print("Australian")
            elif long_array[0] >= 160  and long_array[0] < 300 and lati_array[0] < 0:
                print("South Pacific")
            else:
                print("That's not right!")
            print("\n")

In [ ]:
%%time
### read 
year=2004 ## enter year here. We have years 2002-2013 processed
tcs=hurricance(year,[buoy_path,precip_path])
tcs.extract_tracks()  
tcs.read_data()
internal = 1 #determine the size of an inner box compared to its environment
outter = offset[0] #determine size of environment
timeahead=2 #how far ahead you wish to look in the storm's track to compare
off_check = outter-internal #the values in between the inner and outter
off_range = (2*outter)
#Updates to Include: Find an equation via machine learning to auto calculate the storm's size

In [ ]:
tropic_non_landfall = []

for i in range(len(tropics)):
    for j in range(len(non_landfall)):
        if tropics[i] == non_landfall[j]:
            #so I can coordinate storms that stay within the tropics and don't landfall
            tropic_non_landfall.append(tropics[i])
#print(tropic_non_landfall)

tropic_landfall = []
for i in range(len(tropics)):
    for j in range(len(landfall)):
        if tropics[i] == landfall[j]:
            tropic_landfall.append(tropics[i])
            
print(tropic_landfall)
print(tropic_non_landfall)
tc_number = 52 #choice of tropical cyclone for the graphs below to evaluate them solo
lati_array=tcs.tc_tracks[tc_number]['lat']
long_array=tcs.tc_tracks[tc_number]['lon']
pslmax=np.argmin(tcs.tc_tracks[tc_number]['psl'])
pslmaxval=np.min(tcs.tc_tracks[tc_number]['psl'])
type_of_plot="Buoyancy"
#Choices Include:
#CAPE/Subsaturation
#EQBL/EQSLFT/EQULFT
#Buoyancy
#Precipitation

# Calculates the Average Across the Entire Offset Box

In [ ]:

g = np.int_(np.linspace(0,tcs.ds_comp[tc_number].precipitation.time.size-1)) #so I get the exact number of frames
x = np.arange(0,max(g)+1,1)
eqblwhole = [0] * len(x) #empty array for EQBL
eqslftwhole= [0] * len(x) #empty array for EQSLFT
equlftwhole = [0] * len(x) #empty array for EQULFT
subswhole = [0] * len(x) #empty array for subsaturation
capewhole = [0] * len(x) #empty array for CAPE
buoywhole = [0] * len(x) #empty array for buoyancy
preciwhole = [0] * len(x) #empty array for precipitation 

box_size = [] 
lat_array=tcs.tc_tracks[tc_number]['lat']
lon_array=tcs.tc_tracks[tc_number]['lon']
for t in range(len(lon_array)):
    if lon_array[t]<0:
        lon_array[t]=lon_array[t]+360

 
for t in range(max(g)+1):


    eqbl= tcs.ds_comp[tc_number].EQBL.isel(time=t) #taking the values for equivalent pot. temp. in the ABL
    eqblada=eqbl.stack(x=['lon','lat'])
    eqbla=eqblada[eqblada.notnull()] #so I can manipulate the values for this factor and take the inside and outside

    eqslft= tcs.ds_comp[tc_number].EQSLFT.isel(time=t) #taking the values for equivalent sat. pot. temp. in the LFT
    eqslftada=eqslft.stack(x=['lat','lon'])
    eqslfta=eqslftada[eqslftada.notnull()]

    equlft= tcs.ds_comp[tc_number].EQULFT.isel(time=t) #taking the values for equivalent unsat. pot. temp. in the LFT
    equlftada=equlft.stack(x=['lat','lon'])
    equlfta=equlftada[equlftada.notnull()]

    subs= tcs.ds_comp[tc_number].Subsaturation.isel(time=t) #taking the values for subsaturation in the LFT
    subsada_stacked=subs.stack(x=['lat','lon'])
    subsa=subsada_stacked[subsada_stacked.notnull()]

    cape= tcs.ds_comp[tc_number].CAPE.isel(time=t) #taking the values for CAPE in the LFT
    capada_stacked=cape.stack(x=['lat','lon'])
    capea=capada_stacked[capada_stacked.notnull()]

    buoy= tcs.ds_comp[tc_number].Buoyancy.isel(time=t) #taking the buoyancy for buoyancy in the LFT
    buoyada=buoy.stack(x=['lat','lon'])
    buoya=buoyada[buoyada.notnull()]

    preci= tcs.ds_comp[tc_number].precipitation.isel(time=t) #taking the values for precipitation
    preciada=preci.stack(x=['lat','lon'])
    precia=preciada[preciada.notnull()]
    max_long=lat_array[t]+outter
    min_long=lat_array[t]-outter
    if (min_long > 34.75) or (max_long <-34.75):
        eqblwhole[t]=(float("nan")) #for storms that have no values to calculate (above/below 35 N/S)
        eqslftwhole[t]=(float("nan"))
        equlftwhole[t]=(float("nan"))
        subswhole[t]=(float("nan"))
        capewhole[t]=(float("nan"))
        buoywhole[t]=(float("nan"))
        preciwhole[t]=(float("nan"))
        box_size.append(t)
    else:

        r=lon_array[t]+internal #max longitude inside
        qq=r
        qq_check=r+off_check #max longitude outside


        o=lon_array[t]-internal #min longitude inside
        pp=o
        pp_check=o-off_check #min longitude outside



        rr=lat_array[t]+internal #max latitude inside
        q=rr
        q_check=rr+off_check #max latitude outside

        oo=lat_array[t]-internal #min latitude inside
        p=oo
        p_check=oo-off_check #min latitude outside

        eqbl_check = eqbla.where((eqbla.lon>=pp_check) & (eqbla.lon<=qq_check) & (eqbla.lat>=p_check) & (eqbla.lat<=q_check))
        eqslft_check = eqslfta.where((eqslfta.lon>=pp_check) & (eqslfta.lon<=qq) & (eqslfta.lat>=p_check) & (eqslfta.lat<=q_check))
        equlft_check = equlfta.where((equlfta.lon>=pp_check) & (equlfta.lon<=qq) & (equlfta.lat>=p_check) & (equlfta.lat<=q_check))

        subs_check = subsa.where((subsa.lon>=pp_check) & (subsa.lon<=qq_check) & (subsa.lat>=p_check) & (subsa.lat<=q_check))
        cape_check = capea.where((capea.lon>=pp_check) & (capea.lon<=qq_check) & (capea.lat>=p_check) & (capea.lat<=q_check))

        buoy_check = buoya.where((buoya.lon>=pp) & (buoya.lon<=qq_check) & (buoya.lat>=p_check) & (buoya.lat<=q_check))
        preci_check = precia.where((precia.lon>=pp) & (precia.lon<=qq_check) & (precia.lat>=p_check) & (precia.lat<=q_check))
        if (max_long <= 34.75) & (min_long >=-34.75): #for the box when full sized
            
            box_size.append("x")
        else: #for if the box size becomes less than full
            box_size.append(t)
            

        eqblwhole[t] = np.nanmean(eqbl_check)
        eqslftwhole[t] = np.nanmean(eqslft_check)
        equlftwhole[t] = np.nanmean(equlft_check)
        subswhole[t] = np.nanmean(subs_check)
        capewhole[t] = np.nanmean(cape_check)
        buoywhole[t] = np.nanmean(buoy_check)
        preciwhole[t] = np.nanmean(preci_check)

time_array = []
for t in range(max(g)+1):
    v = tcs.ds_comp[tc_number].time.dt.strftime("%m%d-%H").isel(time=t).item()
    time_array.append(v)
plt.figure(figsize = (10,10))
plt.xticks(x, time_array, rotation = 45)
plt.locator_params(axis='x', nbins=len(x)/4)
if type_of_plot=="EQBL/EQSLFT/EQULFT":
    plt.plot(x,eqblwhole, color=[.7,.1,.7], marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$")
    plt.plot(x,eqslftwhole, color=[.2,.1,.9], marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$")
    plt.plot(x,equlftwhole, color=[.2,.5,.5], marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)

    
elif type_of_plot=="CAPE/Subsaturation":
    plt.plot(x,capewhole, color=[.7,.1,.7], marker = '.', markersize = 9, label = "CAPE$_L$")
    plt.plot(x,subswhole, color=[.2,.1,.9], marker = '.', markersize = 9, label = "Subsaturation$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)

elif type_of_plot=="Buoyancy":
    plt.plot(x,buoywhole, color=[.7,.1,.7], marker = '.', markersize = 9, label = "B$_L$")
    plt.ylabel("Acceleration [$m/s^2$]", fontsize = 10)

    
elif type_of_plot=="Precipitation":
    plt.plot(x,preciwhole, color=[.7,.1,.7], marker = '.', markersize = 9, label = "Precipitation")
    plt.ylabel("Rainfall Amount [In]", fontsize = 10)


tes = []
te = []
for na in eqblwhole:
    if np.isnan(na):
        te.append(na)
    else:
        tes.append(na)
if len(te) != 0:
    leng = len(tes)
    plt.axvline(x=x[leng-1], linestyle ="--", color="k", label = "Storm Passes 35{} N/S, Persists {} More Hours".format(chr(176), 6*len(te)))
breaks = 0 #so the label doesn't repeat
for lin in range(len(eqblwhole)): #so I can show what times a storm box is not full size

    if (type(box_size[lin]) == int) & (breaks != 0): #so the label doesn't repeat
        
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B")
    if (type(box_size[lin]) == int) & (breaks == 0):
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B", label = "Box is Not Full Size")
        breaks = breaks + 1
      
breaker = 0 #so the label doesn't repeat
for lan in range(len(eqblwhole)): #so I can show what time values have a storm on land
    if long_array[lan] >= 180: #to correct the landmask
        long_array[lan] = long_array[lan]-360
    if globe.is_land(lati_array[lan], long_array[lan]) == True:
        if breaker != 0: #so the label doesn't repeat
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5)
        if breaker == 0:
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5, label = "Center Over land")
            breaker = breaker + 1
plt.axvline(x=x[pslmax], linewidth=2,label="Minimal Pressure: {:.4f}".format(pslmaxval))   
plt.grid()
plt.legend()
plt.xlabel("6 Hour increments [Month, Day, Hour]", fontsize = 10)


plt.title('Tropical Cyclone # {:d} in year {:d}: Average {} ($\u00B1${} Degree Box)'.format(tc_number,year,type_of_plot,outter),fontsize=14);

# Calculates the Average of the Internal Values (Determined by Internal Box Size)

In [ ]:
# off_check = offset[0]-internal
# off_range = (2*offset[0])-.25
lat_array=tcs.tc_tracks[tc_number]['lat']
lon_array=tcs.tc_tracks[tc_number]['lon']
for t in range(len(lon_array)):
    if lon_array[t]<0:
        lon_array[t]=lon_array[t]+360
g = np.int_(np.linspace(0,tcs.ds_comp[tc_number].precipitation.time.size-1)) #so I get the exact number of frames
x = np.arange(0,max(g)+1,1)
eqblint = [0] * len(x) #empty array for EQBL
eqslftint= [0] * len(x) #empty array for EQSLFT
equlftint = [0] * len(x) #empty array for EQULFT
subsint = [0] * len(x) #empty array for subsaturation
capeint = [0] * len(x) #empty array for CAPE
buoyint = [0] * len(x) #empty array for buoyancy
preciint = [0] * len(x) #empty array for precipitation 
box_size = []


for t in range(max(g)+1):
        
    
    eqbl= tcs.ds_comp[tc_number].EQBL.isel(time=t) #taking the values for equivalent pot. temp. in the ABL
    eqblada=eqbl.stack(x=['lon','lat'])
    eqbla=eqblada[eqblada.notnull()] #so I can manipulate the values for this factor and take the inside and outside

    eqslft= tcs.ds_comp[tc_number].EQSLFT.isel(time=t) #taking the values for equivalent sat. pot. temp. in the LFT
    eqslftada=eqslft.stack(x=['lat','lon'])
    eqslfta=eqslftada[eqslftada.notnull()]

    equlft= tcs.ds_comp[tc_number].EQULFT.isel(time=t) #taking the values for equivalent unsat. pot. temp. in the LFT
    equlftada=equlft.stack(x=['lat','lon'])
    equlfta=equlftada[equlftada.notnull()]

    subs= tcs.ds_comp[tc_number].Subsaturation.isel(time=t) #taking the values for subsaturation in the LFT
    subsada_stacked=subs.stack(x=['lat','lon'])
    subsa=subsada_stacked[subsada_stacked.notnull()]

    cape= tcs.ds_comp[tc_number].CAPE.isel(time=t) #taking the values for CAPE in the LFT
    capada_stacked=cape.stack(x=['lat','lon'])
    capea=capada_stacked[capada_stacked.notnull()]

    buoy= tcs.ds_comp[tc_number].Buoyancy.isel(time=t) #taking the buoyancy for buoyancy in the LFT
    buoyada=buoy.stack(x=['lat','lon'])
    buoya=buoyada[buoyada.notnull()]

    preci= tcs.ds_comp[tc_number].precipitation.isel(time=t) #taking the values for precipitation
    preciada=preci.stack(x=['lat','lon'])
    precia=preciada[preciada.notnull()]
    max_long=lat_array[t]+outter
    min_long=lat_array[t]-outter
    if (min_long > 34.75) or (max_long <-34.75):
        
        eqblint[t]=(float("nan")) #for storms that have no values to calculate (above/below 35 N/S)
        eqslftint[t]=(float("nan"))
        equlftint[t]=(float("nan"))
        subsint[t]=(float("nan"))
        capeint[t]=(float("nan"))
        buoyint[t]=(float("nan"))
        preciint[t]=(float("nan"))
        box_size.append(t)
    else:

        r=lon_array[t]+internal
        o=lon_array[t]-internal
        pp=o
        qq=r

        rr=lat_array[t]+internal
        oo=lat_array[t]-internal
        p=oo
        q=rr

        eqbl_check = eqbla.where((eqbla.lon>=pp) & (eqbla.lon<=qq) & (eqbla.lat>=p) & (eqbla.lat<=q))
        eqslft_check = eqslfta.where((eqslfta.lon>=pp) & (eqslfta.lon<=qq) & (eqslfta.lat>=p) & (eqslfta.lat<=q))
        equlft_check = equlfta.where((equlfta.lon>=pp) & (equlfta.lon<=qq) & (equlfta.lat>=p) & (equlfta.lat<=q))

        subs_check = subsa.where((subsa.lon>=pp) & (subsa.lon<=qq) & (subsa.lat>=p) & (subsa.lat<=q))
        cape_check = capea.where((capea.lon>=pp) & (capea.lon<=qq) & (capea.lat>=p) & (capea.lat<=q))

        buoy_check = buoya.where((buoya.lon>=pp) & (buoya.lon<=qq) & (buoya.lat>=p) & (buoya.lat<=q))
        preci_check = precia.where((precia.lon>=pp) & (precia.lon<=qq) & (precia.lat>=p) & (precia.lat<=q))
        if (max_long <= 34.75) & (min_long >=-34.75): #for the box when full sized

            box_size.append("x")
        else: 
            #for if the box size becomes less than full
            box_size.append(t)

    eqblint[t] = np.nanmean(eqbl_check)
    eqslftint[t] = np.nanmean(eqslft_check)
    equlftint[t] = np.nanmean(equlft_check)
    subsint[t] = np.nanmean(subs_check)
    capeint[t] = np.nanmean(cape_check)
    buoyint[t] = np.nanmean(buoy_check)
    preciint[t] = np.nanmean(preci_check)


time_array = []
x = np.arange(0,max(g)+1,1)
for t in range(max(g)+1):
    v = tcs.ds_comp[tc_number].time.dt.strftime("%m%d-%H").isel(time=t).item()
    time_array.append(v)
plt.figure(figsize = (10,10))
plt.xticks(x, time_array, rotation = 45)
plt.locator_params(axis='x', nbins=len(x)/4)
if type_of_plot=="EQBL/EQSLFT/EQULFT":
    plt.plot(x,eqblint, color='#1A5276', marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$")
    plt.plot(x,eqslftint, color='#239B56', marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$")
    plt.plot(x,equlftint, color='#873600', marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)
    
elif type_of_plot=="CAPE/Subsaturation":
    plt.plot(x,capeint, color='#1A5276', marker = '.', markersize = 9, label = "CAPE$_L$")
    plt.plot(x,subsint, color='#239B56', marker = '.', markersize = 9, label = "Subsaturation$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)

elif type_of_plot=="Buoyancy":
    plt.plot(x,buoyint, color='#1A5276', marker = '.', markersize = 9, label = "B$_L$")
    plt.ylabel("Acceleration [$m/s^2$]", fontsize = 10)

    
elif type_of_plot=="Precipitation":
    plt.plot(x,preciint, color='#1A5276', marker = '.', markersize = 9, label = "Precipitation")
    plt.ylabel("Rainfall Amount [In]", fontsize = 10)

tes = []
te = []
for na in eqblint:
    if np.isnan(na):
        te.append(na)
    else:
        tes.append(na)
if len(te) != 0:
    leng = len(tes)
    plt.axvline(x=x[leng-1], linestyle ="--", color="k", label = "Storm Passes 35{} N/S, Persists {} More Hours".format(chr(176), 6*len(te)))
breaks = 0 #so the label doesn't repeat
for lin in range(len(eqblint)): #so I can show what times a storm box is not full size

    if (type(box_size[lin]) == int) & (breaks != 0): #so the label doesn't repeat
        
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B")
    if (type(box_size[lin]) == int) & (breaks == 0):
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B", label = "Box is Not Full Size")
        breaks = breaks + 1
        
breaker = 0 #so the label doesn't repeat
for lan in range(len(eqblint)): #so I can show what time values have a storm on land
    if long_array[lan] >= 180: #to correct the landmask
        long_array[lan] = long_array[lan]-360
    if globe.is_land(lati_array[lan], long_array[lan]) == True:
        if breaker != 0: #so the label doesn't repeat
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5)
        if breaker == 0:
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5, label = "Center Over land")
            breaker = breaker + 1
plt.axvline(x=x[pslmax], linewidth=2,label="Minimal Pressure: {:.4f}".format(pslmaxval))   
plt.grid()
plt.legend()
plt.xlabel("6 Hour increments [Month, Day, Hour]", fontsize = 10)


plt.title('Tropical Cyclone # {:d} in year {:d}: Average {} ($\u00B1${} Degree Box)'.format(tc_number,year,type_of_plot,internal),fontsize=14);


# Takes All Values Outside of the Internal Values (Determined by The Internal Box Size)

In [ ]:
off_check = offset[0]-internal
off_range = (2*offset[0])-.25
lat_array=tcs.tc_tracks[tc_number]['lat']
lon_array=tcs.tc_tracks[tc_number]['lon']
g = np.int_(np.linspace(0,tcs.ds_comp[tc_number].precipitation.time.size-1)) #so I get the exact number of frames
x = np.arange(0,max(g)+1,1)
eqblext = [0] * len(x) #empty array for EQBL
eqslftext= [0] * len(x) #empty array for EQSLFT
equlftext = [0] * len(x) #empty array for EQULFT
subsext = [0] * len(x) #empty array for subsaturation
capeext = [0] * len(x) #empty array for CAPE
buoyext = [0] * len(x) #empty array for buoyancy
preciext = [0] * len(x) #empty array for precipitation 
box_size = []


for t in range(max(g)+1):
        
    
    eqbl= tcs.ds_comp[tc_number].EQBL.isel(time=t) #taking the values for equivalent pot. temp. in the ABL
    eqblada=eqbl.stack(x=['lat','lon'])
    eqbla=eqblada[eqblada.notnull()] #so I can manipulate the values for this factor and take the inside and outside

    eqslft= tcs.ds_comp[tc_number].EQSLFT.isel(time=t) #taking the values for equivalent sat. pot. temp. in the LFT
    eqslftada=eqslft.stack(x=['lat','lon'])
    eqslfta=eqslftada[eqslftada.notnull()]

    equlft= tcs.ds_comp[tc_number].EQULFT.isel(time=t) #taking the values for equivalent unsat. pot. temp. in the LFT
    equlftada=equlft.stack(x=['lat','lon'])
    equlfta=equlftada[equlftada.notnull()]

    subs= tcs.ds_comp[tc_number].Subsaturation.isel(time=t) #taking the values for subsaturation in the LFT
    subsada_stacked=subs.stack(x=['lat','lon'])
    subsa=subsada_stacked[subsada_stacked.notnull()]

    cape= tcs.ds_comp[tc_number].CAPE.isel(time=t) #taking the values for CAPE in the LFT
    capada_stacked=cape.stack(x=['lat','lon'])
    capea=capada_stacked[capada_stacked.notnull()]

    buoy= tcs.ds_comp[tc_number].Buoyancy.isel(time=t) #taking the buoyancy for buoyancy in the LFT
    buoyada=buoy.stack(x=['lat','lon'])
    buoya=buoyada[buoyada.notnull()]

    preci= tcs.ds_comp[tc_number].precipitation.isel(time=t) #taking the values for precipitation
    preciada=preci.stack(x=['lat','lon'])
    precia=preciada[preciada.notnull()]
    if len(eqbla.lon.values) == 0:
        
        eqblext[t]=(float("nan")) #for storms that have no values to calculate (above/below 35 N/S)
        eqslftext[t]=(float("nan"))
        equlftext[t]=(float("nan"))
        subsext[t]=(float("nan"))
        capeext[t]=(float("nan"))
        buoyext[t]=(float("nan"))
        preciext[t]=(float("nan"))
        box_size.append(t)
    else:
        r=max(eqbla.lon.values)
        o=min(eqbla.lon.values)
        pp=o+off_check
        qq=r-off_check
        
        eqblaorder=eqbla.sortby(eqbla.lat)
        rr=max(eqblaorder.lat.values)
        oo=min(eqblaorder.lat.values)
        if rr-oo == off_range: #for the box when full sized
            p=oo+off_check
            q=rr-off_check
            eqbl_check = eqbla.where((eqbla.lon>=pp) | (eqbla.lon>qq) | (eqbla.lat>=p) | (eqbla.lat>q))
            eqslft_check = eqslfta.where((eqslfta.lon>=pp) | (eqslfta.lon>qq) | (eqslfta.lat>=p) | (eqslfta.lat>q))
            equlft_check = equlfta.where((equlfta.lon>=pp) | (equlfta.lon>qq) | (equlfta.lat>=p) | (equlfta.lat>q))
            
            subs_check = subsa.where((subsa.lon>=pp) | (subsa.lon>qq) | (subsa.lat>=p) | (subsa.lat>q))
            cape_check = capea.where((capea.lon>=pp) | (capea.lon>qq) | (capea.lat>=p) | (capea.lat>q))
            
            buoy_check = buoya.where((buoya.lon>=pp) | (buoya.lon>qq) | (buoya.lat>=p) | (buoya.lat>q))
            preci_check = precia.where((precia.lon>=pp) | (precia.lon>qq) | (precia.lat>=p) | (precia.lat>q))

            box_size.append("x")
        elif (rr-oo < off_range) & (rr-oo >= (off_check+2.5)) & (lat_array[0] > 0): 
            #for when the box's northern end is less than complete, and cuts part of the top
            p=oo+off_check
            offer = off_range-(rr-oo)
            off_fix=off_check-offer
            q=rr-off_fix
            eqbl_check = eqbla.where((eqbla.lon>=pp) | (eqbla.lon>qq) | (eqbla.lat>=p) | (eqbla.lat>q))
            eqslft_check = eqslfta.where((eqslfta.lon>=pp) | (eqslfta.lon>qq) | (eqslfta.lat>=p) | (eqslfta.lat>q))
            equlft_check = equlfta.where((equlfta.lon>=pp) | (equlfta.lon>qq) | (equlfta.lat>=p) | (equlfta.lat>q))
            
            subs_check = subsa.where((subsa.lon>=pp) | (subsa.lon>qq) | (subsa.lat>=p) | (subsa.lat>q))
            cape_check = capea.where((capea.lon>=pp) | (capea.lon>qq) | (capea.lat>=p) | (capea.lat>q))
            
            buoy_check = buoya.where((buoya.lon>=pp) | (buoya.lon>qq) | (buoya.lat>=p) | (buoya.lat>q))
            preci_check = precia.where((precia.lon>=pp) | (precia.lon>qq) | (precia.lat>=p) | (precia.lat>q))
            box_size.append(t)
        elif (rr-oo > off_check) & (rr-oo < (off_check+2.5)) & (lat_array[0] > 0):
            #for when the box's northern end is less than complete, and cuts into the internal part of the box
            p=oo+off_check
            q=rr
            eqbl_check = eqbla.where((eqbla.lon>=pp) | (eqbla.lon>qq) | (eqbla.lat>=p) | (eqbla.lat>q))
            eqslft_check = eqslfta.where((eqslfta.lon>=pp) | (eqslfta.lon>qq) | (eqslfta.lat>=p) | (eqslfta.lat>q))
            equlft_check = equlfta.where((equlfta.lon>=pp) | (equlfta.lon>qq) | (equlfta.lat>=p) | (equlfta.lat>q))
            
            subs_check = subsa.where((subsa.lon>=pp) | (subsa.lon>qq) | (subsa.lat>=p) | (subsa.lat>q))
            cape_check = capea.where((capea.lon>=pp) | (capea.lon>qq) | (capea.lat>=p) | (capea.lat>q))
            
            buoy_check = buoya.where((buoya.lon>=pp) | (buoya.lon>qq) | (buoya.lat>=p) | (buoya.lat>q))
            preci_check = precia.where((precia.lon>=pp) | (precia.lon>qq) | (precia.lat>=p) | (precia.lat>q))
            box_size.append(t)
        elif (rr-oo < off_range) & (rr-oo >= (off_check+2.5)) & (lat_array[0] < 0):
            #for when the box's southern end is less than complete, and cuts part of the bottom
            offer = off_range-(rr-oo)
            off_fix=off_check-offer
            p=oo+off_fix
            q=rr-off_check
            eqbl_check = eqbla.where((eqbla.lon>=pp) | (eqbla.lon>qq) | (eqbla.lat>=p) | (eqbla.lat>q))
            eqslft_check = eqslfta.where((eqslfta.lon>=pp) | (eqslfta.lon>qq) | (eqslfta.lat>=p) | (eqslfta.lat>q))
            equlft_check = equlfta.where((equlfta.lon>=pp) | (equlfta.lon>qq) | (equlfta.lat>=p) | (equlfta.lat>q))
            
            subs_check = subsa.where((subsa.lon>=pp) | (subsa.lon>qq) | (subsa.lat>=p) | (subsa.lat>q))
            cape_check = capea.where((capea.lon>=pp) | (capea.lon>qq) | (capea.lat>=p) | (capea.lat>q))
            
            buoy_check = buoya.where((buoya.lon>=pp) | (buoya.lon>qq) | (buoya.lat>=p) | (buoya.lat>q))
            preci_check = precia.where((precia.lon>=pp) | (precia.lon>qq) | (precia.lat>=p) | (precia.lat>q))
            box_size.append(t)
        elif (rr-oo > off_check) & (rr-oo < (off_check+2.5)) & (lat_array[0] < 0):
            #for when the box's southern end is less than complete, and cuts into the internal part of the box
            p=oo
            q=rr-off_check
            eqbl_check = eqbla.where((eqbla.lon>=pp) | (eqbla.lon>qq) | (eqbla.lat>=p) | (eqbla.lat>q))
            eqslft_check = eqslfta.where((eqslfta.lon>=pp) | (eqslfta.lon>qq) | (eqslfta.lat>=p) | (eqslfta.lat>q))
            equlft_check = equlfta.where((equlfta.lon>=pp) | (equlfta.lon>qq) | (equlfta.lat>=p) | (equlfta.lat>q))
            
            subs_check = subsa.where((subsa.lon>=pp) | (subsa.lon>qq) | (subsa.lat>=p) | (subsa.lat>q))
            cape_check = capea.where((capea.lon>=pp) | (capea.lon>qq) | (capea.lat>=p) | (capea.lat>q))
            
            buoy_check = buoya.where((buoya.lon>=pp) | (buoya.lon>qq) | (buoya.lat>=p) | (buoya.lat>q))
            preci_check = precia.where((precia.lon>=pp) | (precia.lon>qq) | (precia.lat>=p) | (precia.lat>q))
            box_size.append(t)
        
        else:
            #for when the internal value is cut out
            eqbl_check = eqbla
            eqslft_check = eqslfta
            equlft_check = equlfta
            subs_check = subsa
            cape_check = capea
            buoy_check = buoya
            preci_check = precia
            box_size.append(t)
    eqblext[t] = np.nanmean(eqbl_check)
    eqslftext[t] = np.nanmean(eqslft_check)
    equlftext[t] = np.nanmean(equlft_check)
    subsext[t] = np.nanmean(subs_check)
    capeext[t] = np.nanmean(cape_check)
    buoyext[t] = np.nanmean(buoy_check)
    preciext[t] = np.nanmean(preci_check)




time_array = []
x = np.arange(0,max(g)+1,1)
for t in range(max(g)+1):
    v = tcs.ds_comp[tc_number].time.dt.strftime("%m%d-%H").isel(time=t).item()
    time_array.append(v)
plt.figure(figsize = (10,10))
plt.xticks(x, time_array, rotation = 45)
plt.locator_params(axis='x', nbins=len(x)/4)
if type_of_plot=="EQBL/EQSLFT/EQULFT":
    plt.plot(x,eqblext, color='#F1C40F', marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$")
    plt.plot(x,eqslftext, color='#CB926D', marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$")
    plt.plot(x,equlftext, color='#E74C3C', marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)
    
elif type_of_plot=="CAPE/Subsaturation":
    plt.plot(x,capeext, color='#F1C40F', marker = '.', markersize = 9, label = "CAPE$_L$")
    plt.plot(x,subsext, color='#CB926D', marker = '.', markersize = 9, label = "Subsaturation$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)

elif type_of_plot=="Buoyancy":
    plt.plot(x,buoyext, color='#F1C40F', marker = '.', markersize = 9, label = "B$_L$")
    plt.ylabel("Acceleration [$m/s^2$]", fontsize = 10)

    
elif type_of_plot=="Precipitation":
    plt.plot(x,preciext, color='#F1C40F', marker = '.', markersize = 9, label = "Precipitation")
    plt.ylabel("Rainfall Amount [In]", fontsize = 10)
    
tes = []
te = []
for na in eqblext:
    if np.isnan(na):
        te.append(na)
    else:
        tes.append(na)
if len(te) != 0:
    leng = len(tes)
    plt.axvline(x=x[leng-1], linestyle ="--", color="k", label = "Storm Passes 35{} N/S, Persists {} More Hours".format(chr(176), 6*len(te)))
breaks = 0 #so the label doesn't repeat
for lin in range(len(eqblext)): #so I can show what times a storm box is not full size

    if (type(box_size[lin]) == int) & (breaks != 0): #so the label doesn't repeat
        
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B")
    if (type(box_size[lin]) == int) & (breaks == 0):
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B", label = "Box is Not Full Size")
        breaks = breaks + 1
        
breaker = 0 #so the label doesn't repeat
for lan in range(len(eqblext)): #so I can show what time values have a storm on land
    if long_array[lan] >= 180: #to correct the landmask
        long_array[lan] = long_array[lan]-360
    if globe.is_land(lati_array[lan], long_array[lan]) == True:
        if breaker != 0: #so the label doesn't repeat
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5)
        if breaker == 0:
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5, label = "Center Over land")
            breaker = breaker + 1
plt.axvline(x=x[pslmax], linewidth=2,label="Minimal Pressure: {:.4f}".format(pslmaxval))   
plt.grid()
plt.legend()
plt.xlabel("6 Hour increments [Month, Day, Hour]", fontsize = 10)


plt.title('Tropical Cyclone # {:d} in year {:d}: Average {} ($\u00B1${} (without inner $\u00B1${}) Degree Box)'.format(tc_number,year,type_of_plot,outter,internal),fontsize=14);

# Plots the Internal Values and the External Values

In [ ]:
lat_array=tcs.tc_tracks[tc_number]['lat']
lon_array=tcs.tc_tracks[tc_number]['lon']
for t in range(len(lon_array)):
    if lon_array[t]<0:
        lon_array[t]=lon_array[t]+360
time_array = [] #creating time array so I can include the dates properly
x = np.arange(0,max(g)+1,1)
for t in range(max(g)+1):
    v = tcs.ds_comp[tc_number].time.dt.strftime("%m%d-%H").isel(time=t).item()
    time_array.append(v)
plt.figure(figsize = (10,10))
plt.xticks(x, time_array, rotation = 45)
plt.locator_params(axis='x', nbins=len(x)/4)
if type_of_plot=="EQBL/EQSLFT/EQULFT":
    plt.plot(x,eqblint, color='#1A5276', marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$ inner")
    plt.plot(x,eqslftint, color='#239B56', marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$ inner")
    plt.plot(x,equlftint, color='#873600', marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$ inner")
    plt.plot(x,eqblext, color='#F1C40F', marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$ outter")
    plt.plot(x,eqslftext, color='#CB926D', marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$ outter")
    plt.plot(x,equlftext, color='#E74C3C', marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$ outter")
    plt.ylabel("Degrees [K]", fontsize = 10)


    
    
elif type_of_plot=="CAPE/Subsaturation":
    plt.plot(x,capeint, color='#1A5276', marker = '.', markersize = 9, label = "CAPE$_L$ inner")
    plt.plot(x,subsint, color='#239B56', marker = '.', markersize = 9, label = "Subsaturation$_L$ inner")
    plt.plot(x,capeext, color='#F1C40F', marker = '.', markersize = 9, label = "CAPE$_L$ outter")
    plt.plot(x,subsext, color='#CB926D', marker = '.', markersize = 9, label = "Subsaturation$_L$ outter")
    plt.ylabel("Degrees [K]", fontsize = 10)

elif type_of_plot=="Buoyancy":
    plt.plot(x,buoyint, color='#1A5276', marker = '.', markersize = 9, label = "B$_L$ inner")
    plt.plot(x,buoyext, color='#F1C40F', marker = '.', markersize = 9, label = "B$_L$ outter")
    plt.ylabel("Acceleration [$m/s^2$]", fontsize = 10)

    
elif type_of_plot=="Precipitation":
    plt.plot(x,preciint, color='#1A5276', marker = '.', markersize = 9, label = "Precipitation inner")
    plt.plot(x,preciext, color='#F1C40F', marker = '.', markersize = 9, label = "Precipitation outter")
    plt.ylabel("Rainfall Amount [In]", fontsize = 10)
        

tes = []
te = []
for na in eqblint:
    if np.isnan(na):
        te.append(na)
    else:
        tes.append(na)
if len(te) != 0:
    leng = len(tes)
    plt.axvline(x=x[leng-1], linestyle ="--", color="k", label = "Storm Passes 35{} N/S, Persists {} More Hours".format(chr(176), 6*len(te)))
breaks = 0 #so the label doesn't repeat
for lin in range(len(eqblint)): #so I can show what times a storm box is not full size

    if (type(box_size[lin]) == int) & (breaks != 0): #so the label doesn't repeat
        
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B")
    if (type(box_size[lin]) == int) & (breaks == 0):
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B", label = "Box is Not Full Size")
        breaks = breaks + 1
        
breaker = 0 #so the label doesn't repeat
for lan in range(len(eqblint)): #so I can show what time values have a storm on land
    if long_array[lan] >= 180: #to correct the landmask
        long_array[lan] = long_array[lan]-360
    if globe.is_land(lati_array[lan], long_array[lan]) == True:
        if breaker != 0: #so the label doesn't repeat
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5)
        if breaker == 0:
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5, label = "Center Over land")
            breaker = breaker + 1
plt.axvline(x=x[pslmax], linewidth=2,label="Minimal Pressure: {:.4f}".format(pslmaxval))   
plt.grid()
plt.legend()
plt.xlabel("6 Hour increments [Month, Day, Hour]", fontsize = 10)


plt.title('Tropical Cyclone # {:d} in year {:d}: Average {} ($\u00B1${} and $\u00B1${} (without inner {}))'.format(tc_number,year,type_of_plot,internal,outter,internal),fontsize=14);


# Subtracts the External Average from the Internal Value and Plots

In [ ]:
lat_array=tcs.tc_tracks[tc_number]['lat']
lon_array=tcs.tc_tracks[tc_number]['lon']
for t in range(len(lon_array)):
    if lon_array[t]<0:
        lon_array[t]=lon_array[t]+360
x = np.arange(0,max(g)+1,1)
EQBL_weight_unweight = [0] * len(x) #creating empty array to take the difference between internal and external
for i in x:
    EQBL_weight_unweight[i]= (eqblint[i] - eqblext[i]) #taking the difference between internal and external average
    

EQSLFT_weight_unweight  = [0] * len(x)
for i in x:
    EQSLFT_weight_unweight[i]= (eqslftint[i] - eqslftext[i])


EQULFT_weight_unweight  = [0] * len(x)
for i in x:
    EQULFT_weight_unweight[i]= (equlftint[i] - equlftext[i])

Subsat_weight_unweight = [0] * len(x)
for i in x:
    Subsat_weight_unweight[i]= (subsint[i] - subsext[i])

CAPE_weight_unweight = [0] * len(x)
for i in x:
    CAPE_weight_unweight[i]= (capeint[i] - capeext[i])

Buoy_weight_unweight = [0] * len(x)
for i in x:
    Buoy_weight_unweight[i]= (buoyint[i] - buoyext[i]) 

Precip_weight_unweight = [0] * len(x)
for i in x:
    Precip_weight_unweight[i]= (preciint[i] - preciext[i]) 

time_array = []
#x = np.arange(0,max(g)+1,1)
for t in range(max(g)+1):
    v = tcs.ds_comp[tc_number].time.dt.strftime("%m%d-%H").isel(time=t).item()
    time_array.append(v)
plt.figure(figsize = (10,10))
plt.xticks(x, time_array, rotation = 45)
plt.locator_params(axis='x', nbins=len(x)/4)
if type_of_plot=="EQBL/EQSLFT/EQULFT":
    plt.plot(x,EQBL_weight_unweight, color='r', marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$")
    plt.plot(x,EQSLFT_weight_unweight, color="g", marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$")
    plt.plot(x,EQULFT_weight_unweight, color="indigo", marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)

elif type_of_plot=="CAPE/Subsaturation":
    plt.plot(x,CAPE_weight_unweight, color='r', marker = '.', markersize = 9, label = "CAPE$_L$")
    plt.plot(x,Subsat_weight_unweight, color='g', marker = '.', markersize = 9, label = "Subsaturation$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)

elif type_of_plot=="Buoyancy":
    plt.plot(x,Buoy_weight_unweight, color='r', marker = '.', markersize = 9, label = "B$_L$")
    plt.ylabel("Acceleration [$m/s^2$]", fontsize = 10)

    
elif type_of_plot=="Precipitation":
    plt.plot(x,Precip_weight_unweight, color='r', marker = '.', markersize = 9, label = "Precipitation")
    plt.ylabel("Rainfall Amount [In]", fontsize = 10)

plt.axhline(y=0, linestyle ="--", color="#0CE60A",linewidth=2)
tes = []
te = []
for na in eqblint:
    if np.isnan(na):
        te.append(na)
    else:
        tes.append(na)
if len(te) != 0:
    leng = len(tes)
    plt.axvline(x=x[leng-1], linestyle ="--", color="k", label = "Storm Passes 35{} N/S, Persists {} More Hours".format(chr(176), 6*len(te)))
breaks = 0 #so the label doesn't repeat
for lin in range(len(eqblint)): #so I can show what times a storm box is not full size

    if (type(box_size[lin]) == int) & (breaks != 0): #so the label doesn't repeat
        
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B")
    if (type(box_size[lin]) == int) & (breaks == 0):
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B", label = "Box is Not Full Size")
        breaks = breaks + 1
        
breaker = 0 #so the label doesn't repeat
for lan in range(len(eqblint)): #so I can show what time values have a storm on land
    if long_array[lan] >= 180: #to correct the landmask
        long_array[lan] = long_array[lan]-360
    if globe.is_land(lati_array[lan], long_array[lan]) == True:
        if breaker != 0: #so the label doesn't repeat
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5)
        if breaker == 0:
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5, label = "Center Over land")
            breaker = breaker + 1
plt.axvline(x=x[pslmax], linewidth=2,label="Minimal Pressure: {:.4f}".format(pslmaxval))   
plt.grid()
plt.legend()
plt.xlabel("6 Hour increments [Month, Day, Hour]", fontsize = 10)



plt.title('Tropical Cyclone # {:d} in year {:d}: Average {} ($\u00B1${} - $\u00B1${} (without inner {}) Degree Box)'.format(tc_number,year,type_of_plot,internal,outter,internal),fontsize=14);

# Takes all Previous Graphs From Above and Combines them into a Subplot Group

In [ ]:
plt.subplots(figsize = (15,15))
for t in range(len(lon_array)):
    if lon_array[t]<0:
        lon_array[t]=lon_array[t]+360
plt.subplot(2,2,1)
plt.xticks(x, time_array, rotation = 45)
plt.locator_params(axis='x', nbins=len(x)/4)
if type_of_plot=="EQBL/EQSLFT/EQULFT":
    plt.plot(x,eqblint, color='#1A5276', marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$")
    plt.plot(x,eqslftint, color='#239B56', marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$")
    plt.plot(x,equlftint, color='#873600', marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)
    minim=min(np.nanmin(eqblext),np.nanmin(eqslftext),np.nanmin(equlftext),np.nanmin(eqblint),np.nanmin(eqslftint),np.nanmin(equlftint))
    maxi=max(np.nanmax(eqblext),np.nanmax(eqslftext),np.nanmax(equlftext),np.nanmax(eqblint),np.nanmax(eqslftint),np.nanmax(equlftint))
    plt.ylim(minim-2, maxi+2)
    
elif type_of_plot=="CAPE/Subsaturation":
    plt.plot(x,capeint, color='#1A5276', marker = '.', markersize = 9, label = "CAPE$_L$")
    plt.plot(x,subsint, color='#239B56', marker = '.', markersize = 9, label = "Subsaturation$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)
    minim=min(np.nanmin(capeext),np.nanmin(subsext),np.nanmin(capeint),np.nanmin(subsint))
    maxi=max(np.nanmax(capeext),np.nanmax(subsext),np.nanmax(capeint),np.nanmax(subsint))
    plt.ylim(minim-1, maxi+1)

elif type_of_plot=="Buoyancy":
    plt.plot(x,buoyint, color='#1A5276', marker = '.', markersize = 9, label = "B$_L$")
    plt.ylabel("Acceleration [$m/s^2$]", fontsize = 10)
    minim=min(np.nanmin(buoyext),np.nanmin(buoyint))
    maxi=max(np.nanmax(buoyext),np.nanmax(buoyint))
    plt.ylim(minim-.005, maxi+.005)
    
elif type_of_plot=="Precipitation":
    plt.plot(x,preciint, color='#1A5276', marker = '.', markersize = 9, label = "Precipitation")
    plt.ylabel("Rainfall Amount [In]", fontsize = 10)
    minim=min(np.nanmin(preciext),np.nanmin(preciint))
    maxi=max(np.nanmax(preciext),np.nanmax(preciint))
    plt.ylim(minim-.5, maxi+.5)

tes = []
te = [] #calculates when the storm's center passes 35 N/S

for na in eqblint:
    if np.isnan(na):
        te.append(na)
    else:
        tes.append(na)
if len(te) != 0:
    leng = len(tes)
    plt.axvline(x=x[leng-1], linestyle ="--", color="k", label = "Storm Passes 35{} N/S, Persists {} More Hours".format(chr(176), 6*len(te)))
breaks = 0 #so the label doesn't repeat
for lin in range(len(eqblint)): #so I can show what times a storm box is not full size

    if (type(box_size[lin]) == int) & (breaks != 0): #so the label doesn't repeat
        
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B")
    if (type(box_size[lin]) == int) & (breaks == 0):
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B", label = "Box is Not Full Size")
        breaks = breaks + 1
        
breaker = 0 #so the label doesn't repeat
for lan in range(len(eqblint)): #so I can show what time values have a storm on land
    if long_array[lan] >= 180: #to correct the landmask
        long_array[lan] = long_array[lan]-360
    if globe.is_land(lati_array[lan], long_array[lan]) == True:
        if breaker != 0: #so the label doesn't repeat
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5)
        if breaker == 0:
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5, label = "Center Over land")
            breaker = breaker + 1
plt.axvline(x=x[pslmax], linewidth=2,label="Minimal Pressure: {:.4f}".format(pslmaxval))   
plt.grid()
plt.legend()
plt.xlabel("6 Hour increments [Month, Day, Hour]", fontsize = 10)


plt.title('$\u00B1${} Degree Box'.format(internal),fontsize=12);
for t in range(len(lon_array)):
    if lon_array[t]<0:
        lon_array[t]=lon_array[t]+360
plt.subplot(2,2,2)
plt.xticks(x, time_array, rotation = 45)
plt.locator_params(axis='x', nbins=len(x)/4)
if type_of_plot=="EQBL/EQSLFT/EQULFT":
    plt.plot(x,eqblext, color='#F1C40F', marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$")
    plt.plot(x,eqslftext, color='#CB926D', marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$")
    plt.plot(x,equlftext, color='#E74C3C', marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)
    plt.ylim(minim-2, maxi+2)
elif type_of_plot=="CAPE/Subsaturation":
    plt.plot(x,capeext, color='#F1C40F', marker = '.', markersize = 9, label = "CAPE$_L$")
    plt.plot(x,subsext, color='#CB926D', marker = '.', markersize = 9, label = "Subsaturation$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)
    plt.ylim(minim-1, maxi+1)
elif type_of_plot=="Buoyancy":
    plt.plot(x,buoyext, color='#F1C40F', marker = '.', markersize = 9, label = "B$_L$")
    plt.ylabel("Acceleration [$m/s^2$]", fontsize = 10)
    plt.ylim(minim-.005, maxi+.005)
    
elif type_of_plot=="Precipitation":
    plt.plot(x,preciext, color='#F1C40F', marker = '.', markersize = 9, label = "Precipitation")
    plt.ylabel("Rainfall Amount [In]", fontsize = 10)
    plt.ylim(minim-.5, maxi+.5)
    
tes = [] #calculates when the storm's center passes 35 N/S
te = []
for na in eqblext:
    if np.isnan(na):
        te.append(na)
    else:
        tes.append(na)
if len(te) != 0:
    leng = len(tes)
    plt.axvline(x=x[leng-1], linestyle ="--", color="k", label = "Storm Passes 35{} N/S, Persists {} More Hours".format(chr(176), 6*len(te)))
breaks = 0 #so the label doesn't repeat
for lin in range(len(eqblext)): #so I can show what times a storm box is not full size

    if (type(box_size[lin]) == int) & (breaks != 0): #so the label doesn't repeat
        
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B")
    if (type(box_size[lin]) == int) & (breaks == 0):
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B", label = "Box is Not Full Size")
        breaks = breaks + 1
        
breaker = 0 #so the label doesn't repeat
for lan in range(len(eqblext)): #so I can show what time values have a storm on land
    if long_array[lan] >= 180: #to correct the landmask
        long_array[lan] = long_array[lan]-360
    if globe.is_land(lati_array[lan], long_array[lan]) == True:
        if breaker != 0: #so the label doesn't repeat
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5)
        if breaker == 0:
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5, label = "Center Over land")
            breaker = breaker + 1
plt.axvline(x=x[pslmax], linewidth=2,label="Minimal Pressure: {:.4f}".format(pslmaxval))   
plt.grid()

plt.legend()
plt.xlabel("6 Hour increments [Month, Day, Hour]", fontsize = 10)


plt.title('$\u00B1${} (without inner $\u00B1${}) Degree Box'.format(outter,internal),fontsize=12);
for t in range(len(lon_array)):
    if lon_array[t]<0:
        lon_array[t]=lon_array[t]+360
plt.subplot(2,2,3)

plt.xticks(x, time_array, rotation = 45)
plt.locator_params(axis='x', nbins=len(x)/4)
if type_of_plot=="EQBL/EQSLFT/EQULFT":
    plt.plot(x,eqblint, color='#1A5276', marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$ inner")
    plt.plot(x,eqslftint, color='#239B56', marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$ inner")
    plt.plot(x,equlftint, color='#873600', marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$ inner")
    plt.plot(x,eqblext, color='#F1C40F', marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$ outter")
    plt.plot(x,eqslftext, color='#CB926D', marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$ outter")
    plt.plot(x,equlftext, color='#E74C3C', marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$ outter")
    plt.ylabel("Degrees [K]", fontsize = 10)
    plt.ylim(minim-2, maxi+2)
elif type_of_plot=="CAPE/Subsaturation":
    plt.plot(x,capeint, color='#1A5276', marker = '.', markersize = 9, label = "CAPE$_L$ inner")
    plt.plot(x,subsint, color='#239B56', marker = '.', markersize = 9, label = "Subsaturation$_L$ inner")
    plt.plot(x,capeext, color='#F1C40F', marker = '.', markersize = 9, label = "CAPE$_L$ outter")
    plt.plot(x,subsext, color='#CB926D', marker = '.', markersize = 9, label = "Subsaturation$_L$ outter")
    plt.ylabel("Degrees [K]", fontsize = 10)
    plt.ylim(minim-1, maxi+1)
elif type_of_plot=="Buoyancy":
    plt.plot(x,buoyint, color='#1A5276', marker = '.', markersize = 9, label = "B$_L$ inner")
    plt.plot(x,buoyext, color='#F1C40F', marker = '.', markersize = 9, label = "B$_L$ outter")
    plt.ylabel("Acceleration [$m/s^2$]", fontsize = 10)
    plt.ylim(minim-.005, maxi+.005)
elif type_of_plot=="Precipitation":
    plt.plot(x,preciint, color='#1A5276', marker = '.', markersize = 9, label = "Precipitation inner")
    plt.plot(x,preciext, color='#F1C40F', marker = '.', markersize = 9, label = "Precipitation outter")
    plt.ylabel("Rainfall Amount [In]", fontsize = 10)
    plt.ylim(minim-.5, maxi+.5)
    
tes = [] #calculates when the storm's center passes 35 N/S
te = []
for na in eqblint:
    if np.isnan(na):
        te.append(na)
    else:
        tes.append(na)
if len(te) != 0:
    leng = len(tes)
    plt.axvline(x=x[leng-1], linestyle ="--", color="k", label = "Storm Passes 35{} N/S, Persists {} More Hours".format(chr(176), 6*len(te)))
breaks = 0 #so the label doesn't repeat
for lin in range(len(eqblint)): #so I can show what times a storm box is not full size

    if (type(box_size[lin]) == int) & (breaks != 0): #so the label doesn't repeat
        
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B")
    if (type(box_size[lin]) == int) & (breaks == 0):
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B", label = "Box is Not Full Size")
        breaks = breaks + 1
        
breaker = 0 #so the label doesn't repeat
for lan in range(len(eqblint)): #so I can show what time values have a storm on land
    if long_array[lan] >= 180: #to correct the landmask
        long_array[lan] = long_array[lan]-360
    if globe.is_land(lati_array[lan], long_array[lan]) == True:
        if breaker != 0: #so the label doesn't repeat
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5)
        if breaker == 0:
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5, label = "Center Over land")
            breaker = breaker + 1
plt.axvline(x=x[pslmax], linewidth=2,label="Minimal Pressure: {:.4f}".format(pslmaxval))   
plt.grid()
plt.legend()
plt.xlabel("6 Hour increments [Month, Day, Hour]", fontsize = 10)


plt.title('$\u00B1${} and $\u00B1${} (without inner {})'.format(internal,outter,internal),fontsize=12);
for t in range(len(lon_array)):
    if lon_array[t]<0:
        lon_array[t]=lon_array[t]+360
plt.subplot(2,2,4)
plt.xticks(x, time_array, rotation = 45)
plt.locator_params(axis='x', nbins=len(x)/4)
if type_of_plot=="EQBL/EQSLFT/EQULFT":
    plt.plot(x,EQBL_weight_unweight, color='r', marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$")
    plt.plot(x,EQSLFT_weight_unweight, color="g", marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$")
    plt.plot(x,EQULFT_weight_unweight, color="indigo", marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)

elif type_of_plot=="CAPE/Subsaturation":
    plt.plot(x,CAPE_weight_unweight, color='r', marker = '.', markersize = 9, label = "CAPE$_L$")
    plt.plot(x,Subsat_weight_unweight, color='g', marker = '.', markersize = 9, label = "Subsaturation$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)

elif type_of_plot=="Buoyancy":
    plt.plot(x,Buoy_weight_unweight, color='r', marker = '.', markersize = 9, label = "B$_L$")
    plt.ylabel("Acceleration [$m/s^2$]", fontsize = 10)

    
elif type_of_plot=="Precipitation":
    plt.plot(x,Precip_weight_unweight, color='r', marker = '.', markersize = 9, label = "Precipitation")
    plt.ylabel("Rainfall Amount [In]", fontsize = 10)
plt.axhline(y=0, linestyle ="--", color="#0CE60A",linewidth=2) 
#positive = internal has higher values
#negative = external has higher values
tes = [] #calculates when the storm's center passes 35 N/S
te = []
for na in eqblint:
    if np.isnan(na):
        te.append(na)
    else:
        tes.append(na)
if len(te) != 0:
    leng = len(tes)
    plt.axvline(x=x[leng-1], linestyle ="--", color="k", label = "Storm Passes 35{} N/S, Persists {} More Hours".format(chr(176), 6*len(te)))
breaks = 0 #so the label doesn't repeat
for lin in range(len(eqblint)): #so I can show what times a storm box is not full size

    if (type(box_size[lin]) == int) & (breaks != 0): #so the label doesn't repeat
        
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B")
    if (type(box_size[lin]) == int) & (breaks == 0):
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B", label = "Box is Not Full Size")
        breaks = breaks + 1
        
breaker = 0 #so the label doesn't repeat
for lan in range(len(eqblint)): #so I can show what time values have a storm on land
    if long_array[lan] >= 180: #to correct the landmask
        long_array[lan] = long_array[lan]-360
    if globe.is_land(lati_array[lan], long_array[lan]) == True:
        if breaker != 0: #so the label doesn't repeat
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5)
        if breaker == 0:
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5, label = "Center Over land")
            breaker = breaker + 1
plt.axvline(x=x[pslmax], linewidth=2,label="Minimal Pressure: {:.4f}".format(pslmaxval))   
plt.grid()
plt.legend()
plt.xlabel("6 Hour increments [Month, Day, Hour]", fontsize = 10)


plt.title('$\u00B1${} - $\u00B1${} (without inner {}) Degree Box'.format(internal,outter,internal),fontsize=12);
plt.suptitle('Tropical Cyclone # {:d} in year {:d}: {}'.format(tc_number,year,type_of_plot),fontsize=18);
plt.tight_layout()

# Take Average Values X Time Steps Ahead of Storm Position

In [ ]:

lat_array=tcs.tc_tracks[tc_number]['lat']
lon_array=tcs.tc_tracks[tc_number]['lon']
for t in range(len(lon_array)):
    if lon_array[t]<0:
        lon_array[t]=lon_array[t]+360
g = np.int_(np.linspace(0,tcs.ds_comp[tc_number].precipitation.time.size-1)) #so I get the exact number of frames
newset=max(g)+1-timeahead #for the range of values
z = np.arange(0,newset,1)
eqblahead = [0] * len(z) #empty array for EQBL
eqslftahead= [0] * len(z) #empty array for EQSLFT
equlftahead = [0] * len(z) #empty array for EQULFT
subsahead = [0] * len(z) #empty array for subsaturation
capeahead = [0] * len(z) #empty array for CAPE
buoyahead = [0] * len(z) #empty array for buoyancy
preciahead = [0] * len(z) #empty array for precipitation 
boxer = []

for t in range(newset):
        
    
    eqbl= tcs.ds_comp[tc_number].EQBL.isel(time=t) #taking the values for equivalent pot. temp. in the ABL
    eqblada=eqbl.stack(x=['lon','lat'])
    eqbla=eqblada[eqblada.notnull()] #so I can manipulate the values for this factor and take the inside and outside

    eqslft= tcs.ds_comp[tc_number].EQSLFT.isel(time=t) #taking the values for equivalent sat. pot. temp. in the LFT
    eqslftada=eqslft.stack(x=['lat','lon'])
    eqslfta=eqslftada[eqslftada.notnull()]

    equlft= tcs.ds_comp[tc_number].EQULFT.isel(time=t) #taking the values for equivalent unsat. pot. temp. in the LFT
    equlftada=equlft.stack(x=['lat','lon'])
    equlfta=equlftada[equlftada.notnull()]

    subs= tcs.ds_comp[tc_number].Subsaturation.isel(time=t) #taking the values for subsaturation in the LFT
    subsada_stacked=subs.stack(x=['lat','lon'])
    subsa=subsada_stacked[subsada_stacked.notnull()]

    cape= tcs.ds_comp[tc_number].CAPE.isel(time=t) #taking the values for CAPE in the LFT
    capada_stacked=cape.stack(x=['lat','lon'])
    capea=capada_stacked[capada_stacked.notnull()]

    buoy= tcs.ds_comp[tc_number].Buoyancy.isel(time=t) #taking the buoyancy for buoyancy in the LFT
    buoyada=buoy.stack(x=['lat','lon'])
    buoya=buoyada[buoyada.notnull()]

    preci= tcs.ds_comp[tc_number].precipitation.isel(time=t) #taking the values for precipitation
    preciada=preci.stack(x=['lat','lon'])
    precia=preciada[preciada.notnull()]
    max_long=lat_array[t+timeahead]+outter
    min_long=lat_array[t+timeahead]-outter
    if (min_long > 34.75) or (max_long <-34.75):
        
        eqblahead[t]=(float("nan")) #for storms that have no values to calculate (above/below 35 N/S)
        eqslftahead[t]=(float("nan"))
        equlftahead[t]=(float("nan"))
        subsahead[t]=(float("nan"))
        capeahead[t]=(float("nan"))
        buoyahead[t]=(float("nan"))
        preciahead[t]=(float("nan"))
        box_size.append(t)
    else:

        r=lon_array[t+timeahead]+internal
        o=lon_array[t+timeahead]-internal
        pp=o
        qq=r
        rr=lat_array[t+timeahead]+internal
        oo=lat_array[t+timeahead]-internal
        p=oo
        q=rr
        eqbl_check = eqbla.where((eqbla.lon>=pp) & (eqbla.lon<=qq) & (eqbla.lat>=p) & (eqbla.lat<=q))
        eqslft_check = eqslfta.where((eqslfta.lon>=pp) & (eqslfta.lon<=qq) & (eqslfta.lat>=p) & (eqslfta.lat<=q))
        equlft_check = equlfta.where((equlfta.lon>=pp) & (equlfta.lon<=qq) & (equlfta.lat>=p) & (equlfta.lat<=q))

        subs_check = subsa.where((subsa.lon>=pp) & (subsa.lon<=qq) & (subsa.lat>=p) & (subsa.lat<=q))
        cape_check = capea.where((capea.lon>=pp) & (capea.lon<=qq) & (capea.lat>=p) & (capea.lat<=q))

        buoy_check = buoya.where((buoya.lon>=pp) & (buoya.lon<=qq) & (buoya.lat>=p) & (buoya.lat<=q))
        preci_check = precia.where((precia.lon>=pp) & (precia.lon<=qq) & (precia.lat>=p) & (precia.lat<=q))
        if (max_long <= 34.75) & (min_long >=-34.75): #for the box when full sized

            boxer.append("x")
        else: 
            #for if the box size becomes less than full
            boxer.append(t)

    eqblahead[t] = np.nanmean(eqbl_check)
    eqslftahead[t] = np.nanmean(eqslft_check)
    equlftahead[t] = np.nanmean(equlft_check)
    subsahead[t] = np.nanmean(subs_check)
    capeahead[t] = np.nanmean(cape_check)
    buoyahead[t] = np.nanmean(buoy_check)
    preciahead[t] = np.nanmean(preci_check)


time_array = []
z = np.arange(0,max(g)-1,1)
for t in np.arange(0,max(g)-1,1):
    v = tcs.ds_comp[tc_number].time.dt.strftime("%m%d-%H").isel(time=t).item()
    time_array.append(v)
plt.figure(figsize = (10,10))
plt.xticks(z, time_array, rotation = 45)
plt.locator_params(axis='x', nbins=len(x)/4)
if type_of_plot=="EQBL/EQSLFT/EQULFT":
    plt.plot(z,eqblahead, color='#84A772', marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$")
    plt.plot(z,eqslftahead, color='#E74C3C', marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$")
    plt.plot(z,equlftahead, color='b', marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)
    
elif type_of_plot=="CAPE/Subsaturation":
    plt.plot(z,capeahead, color='#84A772', marker = '.', markersize = 9, label = "CAPE$_L$")
    plt.plot(z,subsahead, color='#E74C3C', marker = '.', markersize = 9, label = "Subsaturation$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)

elif type_of_plot=="Buoyancy":
    plt.plot(z,buoyahead, color='#84A772', marker = '.', markersize = 9, label = "B$_L$")
    plt.ylabel("Acceleration [$m/s^2$]", fontsize = 10)

    
elif type_of_plot=="Precipitation":
    plt.plot(z,preciahead, color='#84A772', marker = '.', markersize = 9, label = "Precipitation")
    plt.ylabel("Rainfall Amount [In]", fontsize = 10)

tes = []
te = []
for na in eqblahead:
    if np.isnan(na):
        te.append(na)
    else:
        tes.append(na)
if len(te) != 0:
    leng = len(tes)
    plt.axvline(x=x[leng-1], linestyle ="--", color="k", label = "Storm Passes 35{} N/S, Persists {} More Hours".format(chr(176), 6*len(te)))
breaks = 0 #so the label doesn't repeat
for lin in range(len(eqblahead)): #so I can show what times a storm box is not full size

    if (type(boxer[lin]) == int) & (breaks != 0): #so the label doesn't repeat
        
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B")
    if (type(boxer[lin]) == int) & (breaks == 0):
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B", label = "Box is Not Full Size")
        breaks = breaks + 1
        
breaker = 0 #so the label doesn't repeat
for lan in range(len(eqblahead)): #so I can show what time values have a storm on land
    if long_array[lan] >= 180: #to correct the landmask
        long_array[lan] = long_array[lan]-360
    if globe.is_land(lati_array[lan], long_array[lan]) == True:
        if breaker != 0: #so the label doesn't repeat
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5)
        if breaker == 0:
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5, label = "Center Over land")
            breaker = breaker + 1
plt.axvline(x=x[pslmax], linewidth=2,label="Minimal Pressure: {:.4f}".format(pslmaxval))   
plt.grid()
plt.legend()
plt.xlabel("Time Step in Storm's Life", fontsize = 10)


plt.title('Tropical Cyclone # {:d} in year {:d}: Average {} ($\u00B1${} Degree Box) Environment {} Timesteps Ahead'.format(tc_number,year,type_of_plot,internal,timeahead),fontsize=14);


# Plots the Internal Values and the Ahead Values

In [ ]:
for t in range(len(lon_array)):
    if lon_array[t]<0:
        lon_array[t]=lon_array[t]+360
time_array = []
corr = np.arange(timeahead,newset+timeahead,1)
x = np.arange(0,max(g)+1,1)
for t in range(max(g)+1):
    v = tcs.ds_comp[tc_number].time.dt.strftime("%m%d-%H").isel(time=t).item()
    time_array.append(v)
plt.figure(figsize = (10,10))
plt.xticks(x, time_array, rotation = 45)
plt.locator_params(axis='x', nbins=len(x)/4)
if type_of_plot=="EQBL/EQSLFT/EQULFT":
    plt.plot(x,eqblint, color='#1A5276', marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$ inner")
    plt.plot(x,eqslftint, color='#239B56', marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$ inner")
    plt.plot(x,equlftint, color='#873600', marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$ inner")
    plt.plot(corr,eqblahead, color='#84A772', marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$ ahead")
    plt.plot(corr,eqslftahead, color='#E74C3C', marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$ ahead")
    plt.plot(corr,equlftahead, color='b', marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$ ahead")
    plt.ylabel("Degrees [K]", fontsize = 10)

elif type_of_plot=="CAPE/Subsaturation":
    plt.plot(x,capeint, color='#1A5276', marker = '.', markersize = 9, label = "CAPE$_L$ inner")
    plt.plot(x,subsint, color='#239B56', marker = '.', markersize = 9, label = "Subsaturation$_L$ inner")
    plt.plot(corr,capeahead, color='#84A772', marker = '.', markersize = 9, label = "CAPE$_L$ ahead")
    plt.plot(corr,subsahead, color='#E74C3C', marker = '.', markersize = 9, label = "Subsaturation$_L$ ahead")
    plt.ylabel("Degrees [K]", fontsize = 10)

elif type_of_plot=="Buoyancy":
    plt.plot(x,buoyint, color='#1A5276', marker = '.', markersize = 9, label = "B$_L$ inner")
    plt.plot(corr,buoyahead, color='#84A772', marker = '.', markersize = 9, label = "B$_L$ ahead")
    plt.ylabel("Acceleration [$m/s^2$]", fontsize = 10)

elif type_of_plot=="Precipitation":
    plt.plot(x,preciint, color='#1A5276', marker = '.', markersize = 9, label = "Precipitation inner")
    plt.plot(corr,preciahead, color='#84A772', marker = '.', markersize = 9, label = "Precipitation ahead")
    plt.ylabel("Rainfall Amount [In]", fontsize = 10)

    
tes = [] #calculates when the storm's center passes 35 N/S
te = []
for na in eqblint:
    if np.isnan(na):
        te.append(na)
    else:
        tes.append(na)
if len(te) != 0:
    leng = len(tes)
    plt.axvline(x=x[leng-1], linestyle ="--", color="k", label = "Storm Passes 35{} N/S, Persists {} More Hours".format(chr(176), 6*len(te)))
breaks = 0 #so the label doesn't repeat
for lin in range(len(eqblint)): #so I can show what times a storm box is not full size

    if (type(box_size[lin]) == int) & (breaks != 0): #so the label doesn't repeat
        
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B")
    if (type(box_size[lin]) == int) & (breaks == 0):
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B", label = "Box is Not Full Size")
        breaks = breaks + 1
        
breaker = 0 #so the label doesn't repeat
for lan in range(len(eqblint)): #so I can show what time values have a storm on land
    if long_array[lan] >= 180: #to correct the landmask
        long_array[lan] = long_array[lan]-360
    if globe.is_land(lati_array[lan], long_array[lan]) == True:
        if breaker != 0: #so the label doesn't repeat
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5)
        if breaker == 0:
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5, label = "Center Over land")
            breaker = breaker + 1
plt.axvline(x=x[pslmax], linewidth=2,label="Minimal Pressure: {:.4f}".format(pslmaxval))   
plt.grid()
plt.legend()
plt.xlabel("6 Hour increments [Month, Day, Hour]", fontsize = 10)


plt.title('Tropical Cyclone # {:d} in year {:d}: $\u00B1${} and Environment {} Time Steps Ahead'.format(tc_number,year,internal,timeahead),fontsize=12);

# Calculate the Difference of Averages Between Internal Values and The Environment Ahead of the Storm

In [ ]:
z = np.arange(0,newset,1)
corr = np.arange(timeahead,newset+timeahead,1)
for t in range(len(lon_array)):
    if lon_array[t]<0:
        lon_array[t]=lon_array[t]+360
eqblnow = [0] * len(z) #empty array for EQBL
eqslftnow= [0] * len(z) #empty array for EQSLFT
equlftnow = [0] * len(z) #empty array for EQULFT
subsnow = [0] * len(z) #empty array for subsaturation
capenow = [0] * len(z) #empty array for CAPE
buoynow = [0] * len(z) #empty array for buoyancy
precinow = [0] * len(z) #empty array for precipitation
for t in np.arange(0,newset,1): #taking the storm with the environment minus the in the previous envrionment
    eqblnow[t]=eqblint[t+timeahead]-eqblahead[t]
    eqslftnow[t]=eqslftint[t+timeahead]-eqslftahead[t]
    equlftnow[t]=equlftint[t+timeahead]-equlftahead[t]
    subsnow[t]=subsint[t+timeahead]-subsahead[t]
    capenow[t]=capeint[t+timeahead]-capeahead[t]
    buoynow[t]=buoyint[t+timeahead]-buoyahead[t]
    precinow[t]=preciint[t+timeahead]-preciahead[t]
timefor_array = []

for t in np.arange(timeahead,max(g)+1,1): #due to haveing a unique time array
    v = tcs.ds_comp[tc_number].time.dt.strftime("%m%d-%H").isel(time=t).item()
    timefor_array.append(v)
    
plt.figure(figsize = (10,10))
plt.locator_params(axis='x', nbins=len(corr)/4)
if type_of_plot=="EQBL/EQSLFT/EQULFT":
    plt.plot(corr,eqblnow, color='r', marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$")
    plt.plot(corr,eqslftnow, color="g", marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$")
    plt.plot(corr,equlftnow, color="indigo", marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)

elif type_of_plot=="CAPE/Subsaturation":
    plt.plot(corr,capenow, color='r', marker = '.', markersize = 9, label = "CAPE$_L$")
    plt.plot(corr,subsnow, color='g', marker = '.', markersize = 9, label = "Subsaturation$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)

elif type_of_plot=="Buoyancy":
    plt.plot(corr,buoynow, color='r', marker = '.', markersize = 9, label = "B$_L$")
    plt.ylabel("Acceleration [$m/s^2$]", fontsize = 10)

    
elif type_of_plot=="Precipitation":
    plt.plot(corr,precinow, color='r', marker = '.', markersize = 9, label = "Precipitation")
    plt.ylabel("Rainfall Amount [In]", fontsize = 10)

plt.axhline(y=0, linestyle ="--", color="#0CE60A",linewidth=2)
tes = []
te = []
for na in eqblahead:
    if np.isnan(na):
        te.append(na)
    else:
        tes.append(na)
if len(te) != 0:
    leng = len(tes)
    plt.axvline(x=x[leng-1]+timeahead, linestyle ="--", color="k", label = "Storm Passes 35{} N/S, Persists {} More Hours".format(chr(176), 6*len(te)))
breaks = 0 #so the label doesn't repeat
for lin in corr: #so I can show what times a storm box is not full size

    if (type(box_size[lin]) == int) & (breaks != 0): #so the label doesn't repeat
        
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B")
    if (type(box_size[lin]) == int) & (breaks == 0):
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B", label = "Box is Not Full Size")
        breaks = breaks + 1
        
breaker = 0 #so the label doesn't repeat
for lan in corr: #so I can show what time values have a storm on land
    if long_array[lan] >= 180: #to correct the landmask
        long_array[lan] = long_array[lan]-360
    if globe.is_land(lati_array[lan], long_array[lan]) == True:
        if breaker != 0: #so the label doesn't repeat
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5)
        if breaker == 0:
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5, label = "Center Over land")
            breaker = breaker + 1
plt.axvline(x=x[pslmax], linewidth=2,label="Minimal Pressure: {:.4f}".format(pslmaxval))   
plt.grid()
plt.legend()
plt.xlabel("Time Step in Storm's Life", fontsize = 10)



plt.title('Tropical Cyclone # {:d} in year {:d}: Average {} ($\u00B1${} - Environment {} Timesteps Ahead)'.format(tc_number,year,type_of_plot,internal,timeahead),fontsize=14);

# Take Ahead Plots and The Internal Value Plots and Create One Subplot

In [ ]:
plt.subplots(figsize = (15,15))
z = np.arange(0,newset,1)
g = np.int_(np.linspace(0,tcs.ds_comp[tc_number].precipitation.time.size-1)) #so I get the exact number of frames
x = np.arange(0,max(g)+1,1)
corr = np.arange(timeahead,newset+timeahead,1)
time_array=[]
for t in range(max(g)+1):
    v = tcs.ds_comp[tc_number].time.dt.strftime("%m%d-%H").isel(time=t).item()
    time_array.append(v)
for t in range(len(lon_array)):
    if lon_array[t]<0:
        lon_array[t]=lon_array[t]+360
plt.subplot(2,2,1)
plt.xticks(x, time_array, rotation = 45)
plt.locator_params(axis='x', nbins=len(x)/4)
if type_of_plot=="EQBL/EQSLFT/EQULFT":
    plt.plot(x,eqblint, color='#1A5276', marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$")
    plt.plot(x,eqslftint, color='#239B56', marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$")
    plt.plot(x,equlftint, color='#873600', marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)
    minim=min(np.nanmin(eqblahead),np.nanmin(eqslftahead),np.nanmin(equlftahead),np.nanmin(eqblint),np.nanmin(eqslftint),np.nanmin(equlftint))
    maxi=max(np.nanmax(eqblahead),np.nanmax(eqslftahead),np.nanmax(equlftahead),np.nanmax(eqblint),np.nanmax(eqslftint),np.nanmax(equlftint))
    plt.ylim(minim-2, maxi+2)
    
elif type_of_plot=="CAPE/Subsaturation":
    plt.plot(x,capeint, color='#1A5276', marker = '.', markersize = 9, label = "CAPE$_L$")
    plt.plot(x,subsint, color='#239B56', marker = '.', markersize = 9, label = "Subsaturation$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)
    minim=min(np.nanmin(capeahead),np.nanmin(subsahead),np.nanmin(capeint),np.nanmin(subsint))
    maxi=max(np.nanmax(capeahead),np.nanmax(subsahead),np.nanmax(capeint),np.nanmax(subsint))
    plt.ylim(minim-1, maxi+1)

elif type_of_plot=="Buoyancy":
    plt.plot(x,buoyint, color='#1A5276', marker = '.', markersize = 9, label = "B$_L$")
    plt.ylabel("Acceleration [$m/s^2$]", fontsize = 10)
    minim=min(np.nanmin(buoyahead),np.nanmin(buoyint))
    maxi=max(np.nanmax(buoyahead),np.nanmax(buoyint))
    plt.ylim(minim-.005, maxi+.005)
    
elif type_of_plot=="Precipitation":
    plt.plot(x,preciint, color='#1A5276', marker = '.', markersize = 9, label = "Precipitation")
    plt.ylabel("Rainfall Amount [In]", fontsize = 10)
    minim=min(np.nanmin(preciahead),np.nanmin(preciint))
    maxi=max(np.nanmax(preciahead),np.nanmax(preciint))
    plt.ylim(minim-.5, maxi+.5)

tes = []
te = [] #calculates when the storm's center passes 35 N/S

for na in eqblint:
    if np.isnan(na):
        te.append(na)
    else:
        tes.append(na)
if len(te) != 0:
    leng = len(tes)
    plt.axvline(x=x[leng-1], linestyle ="--", color="k", label = "Storm Passes 35{} N/S, Persists {} More Hours".format(chr(176), 6*len(te)))
breaks = 0 #so the label doesn't repeat
for lin in range(len(eqblint)): #so I can show what times a storm box is not full size

    if (type(box_size[lin]) == int) & (breaks != 0): #so the label doesn't repeat
        
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B")
    if (type(box_size[lin]) == int) & (breaks == 0):
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B", label = "Box is Not Full Size")
        breaks = breaks + 1
        
breaker = 0 #so the label doesn't repeat
for lan in range(len(eqblint)): #so I can show what time values have a storm on land
    if long_array[lan] >= 180: #to correct the landmask
        long_array[lan] = long_array[lan]-360
    if globe.is_land(lati_array[lan], long_array[lan]) == True:
        if breaker != 0: #so the label doesn't repeat
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5)
        if breaker == 0:
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5, label = "Center Over land")
            breaker = breaker + 1
plt.axvline(x=x[pslmax], linewidth=2,label="Minimal Pressure: {:.4f}".format(pslmaxval))   
plt.grid()

plt.legend()
plt.xlabel("6 Hour increments [Month, Day, Hour]", fontsize = 10)


plt.title('$\u00B1${} Degree Box'.format(internal),fontsize=12);
for t in range(len(lon_array)):
    if lon_array[t]<0:
        lon_array[t]=lon_array[t]+360
plt.subplot(2,2,2)
plt.xticks(x, time_array, rotation = 45)
plt.locator_params(axis='x', nbins=len(x)/4)
if type_of_plot=="EQBL/EQSLFT/EQULFT":
    plt.plot(corr,eqblahead, color='#84A772', marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$")
    plt.plot(corr,eqslftahead, color='#E74C3C', marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$")
    plt.plot(corr,equlftahead, color='b', marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)
    plt.ylim(minim-2, maxi+2)
elif type_of_plot=="CAPE/Subsaturation":
    plt.plot(corr,capeahead, color='#84A772', marker = '.', markersize = 9, label = "CAPE$_L$")
    plt.plot(corr,subsahead, color='#E74C3C', marker = '.', markersize = 9, label = "Subsaturation$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)
    plt.ylim(minim-1, maxi+1)
elif type_of_plot=="Buoyancy":
    plt.plot(corr,buoyahead, color='#84A772', marker = '.', markersize = 9, label = "B$_L$")
    plt.ylabel("Acceleration [$m/s^2$]", fontsize = 10)
    plt.ylim(minim-.005, maxi+.005)
    
elif type_of_plot=="Precipitation":
    plt.plot(corr,preciahead, color='#84A772', marker = '.', markersize = 9, label = "Precipitation")
    plt.ylabel("Rainfall Amount [In]", fontsize = 10)
    plt.ylim(minim-.5, maxi+.5)
tes = [] #calculates when the storm's center passes 35 N/S
te = []
for na in eqblahead:
    if np.isnan(na):
        te.append(na)
    else:
        tes.append(na)
if len(te) != 0:
    leng = len(tes)
    plt.axvline(x=x[leng-1], linestyle ="--", color="k", label = "Storm Passes 35{} N/S, Persists {} More Hours".format(chr(176), 6*len(te)))
breaks = 0 #so the label doesn't repeat
for lin in range(len(eqblahead)): #so I can show what times a storm box is not full size

    if (type(boxer[lin]) == int) & (breaks != 0): #so the label doesn't repeat
        
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B")
    if (type(boxer[lin]) == int) & (breaks == 0):
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B", label = "Box is Not Full Size")
        breaks = breaks + 1
        
breaker = 0 #so the label doesn't repeat
for lan in range(len(eqblahead)): #so I can show what time values have a storm on land
    if long_array[lan] >= 180: #to correct the landmask
        long_array[lan] = long_array[lan]-360
    if globe.is_land(lati_array[lan], long_array[lan]) == True:
        if breaker != 0: #so the label doesn't repeat
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5)
        if breaker == 0:
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5, label = "Center Over land")
            breaker = breaker + 1
plt.axvline(x=x[pslmax], linewidth=2,label="Minimal Pressure: {:.4f}".format(pslmaxval))   
plt.grid()

plt.legend()
plt.xlabel("6 Hour increments [Month, Day, Hour]", fontsize = 10)


plt.title('$\u00B1${} Degree Box Environment {} Time Steps Ahead'.format(internal,timeahead),fontsize=12);
for t in range(len(lon_array)):
    if lon_array[t]<0:
        lon_array[t]=lon_array[t]+360
plt.subplot(2,2,3)

plt.xticks(x, time_array, rotation = 45)
plt.locator_params(axis='x', nbins=len(x)/4)
if type_of_plot=="EQBL/EQSLFT/EQULFT":
    plt.plot(x,eqblint, color='#1A5276', marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$ inner")
    plt.plot(x,eqslftint, color='#239B56', marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$ inner")
    plt.plot(x,equlftint, color='#873600', marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$ inner")
    plt.plot(corr,eqblahead, color='#84A772', marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$ ahead")
    plt.plot(corr,eqslftahead, color='#E74C3C', marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$ ahead")
    plt.plot(corr,equlftahead, color='b', marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$ ahead")
    plt.ylabel("Degrees [K]", fontsize = 10)
    plt.ylim(minim-2, maxi+2)
elif type_of_plot=="CAPE/Subsaturation":
    plt.plot(x,capeint, color='#1A5276', marker = '.', markersize = 9, label = "CAPE$_L$ inner")
    plt.plot(x,subsint, color='#239B56', marker = '.', markersize = 9, label = "Subsaturation$_L$ inner")
    plt.plot(corr,capeahead, color='#84A772', marker = '.', markersize = 9, label = "CAPE$_L$ ahead")
    plt.plot(corr,subsahead, color='#E74C3C', marker = '.', markersize = 9, label = "Subsaturation$_L$ ahead")
    plt.ylabel("Degrees [K]", fontsize = 10)
    plt.ylim(minim-1, maxi+1)
elif type_of_plot=="Buoyancy":
    plt.plot(x,buoyint, color='#1A5276', marker = '.', markersize = 9, label = "B$_L$ inner")
    plt.plot(corr,buoyahead, color='#84A772', marker = '.', markersize = 9, label = "B$_L$ ahead")
    plt.ylabel("Acceleration [$m/s^2$]", fontsize = 10)
    plt.ylim(minim-.005, maxi+.005)
elif type_of_plot=="Precipitation":
    plt.plot(x,preciint, color='#1A5276', marker = '.', markersize = 9, label = "Precipitation inner")
    plt.plot(corr,preciahead, color='#84A772', marker = '.', markersize = 9, label = "Precipitation ahead")
    plt.ylabel("Rainfall Amount [In]", fontsize = 10)
    plt.ylim(minim-.5, maxi+.5)
    
tes = [] #calculates when the storm's center passes 35 N/S
te = []
for na in eqblint:
    if np.isnan(na):
        te.append(na)
    else:
        tes.append(na)
if len(te) != 0:
    leng = len(tes)
    plt.axvline(x=x[leng-1], linestyle ="--", color="k", label = "Storm Passes 35{} N/S, Persists {} More Hours".format(chr(176), 6*len(te)))
breaks = 0 #so the label doesn't repeat
for lin in range(len(eqblint)): #so I can show what times a storm box is not full size

    if (type(box_size[lin]) == int) & (breaks != 0): #so the label doesn't repeat
        
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B")
    if (type(box_size[lin]) == int) & (breaks == 0):
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B", label = "Box is Not Full Size")
        breaks = breaks + 1
        
breaker = 0 #so the label doesn't repeat
for lan in range(len(eqblint)): #so I can show what time values have a storm on land
    if long_array[lan] >= 180: #to correct the landmask
        long_array[lan] = long_array[lan]-360
    if globe.is_land(lati_array[lan], long_array[lan]) == True:
        if breaker != 0: #so the label doesn't repeat
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5)
        if breaker == 0:
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5, label = "Center Over land")
            breaker = breaker + 1
plt.axvline(x=x[pslmax], linewidth=2,label="Minimal Pressure: {:.4f}".format(pslmaxval))   
plt.grid()
plt.legend()
plt.xlabel("6 Hour increments [Month, Day, Hour]", fontsize = 10)


plt.title('$\u00B1${} and Environment {} Time Steps Ahead'.format(internal,timeahead),fontsize=12);
for t in range(len(lon_array)):
    if lon_array[t]<0:
        lon_array[t]=lon_array[t]+360
plt.subplot(2,2,4)
plt.locator_params(axis='x', nbins=len(x)/4)
if type_of_plot=="EQBL/EQSLFT/EQULFT":
    plt.plot(corr,eqblnow, color='r', marker = '.', markersize = 9, label = "$\\theta$$_e$$_B$")
    plt.plot(corr,eqslftnow, color="g", marker = '.', markersize = 9, label = "$\\theta$*$_e$$_L$")
    plt.plot(corr,equlftnow, color="indigo", marker = '.', markersize = 9, label = "$\\theta$$_e$$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)

elif type_of_plot=="CAPE/Subsaturation":
    plt.plot(corr,capenow, color='r', marker = '.', markersize = 9, label = "CAPE$_L$")
    plt.plot(corr,subsnow, color='g', marker = '.', markersize = 9, label = "Subsaturation$_L$")
    plt.ylabel("Degrees [K]", fontsize = 10)

elif type_of_plot=="Buoyancy":
    plt.plot(corr,buoynow, color='r', marker = '.', markersize = 9, label = "B$_L$")
    plt.ylabel("Acceleration [$m/s^2$]", fontsize = 10)

    
elif type_of_plot=="Precipitation":
    plt.plot(corr,precinow, color='r', marker = '.', markersize = 9, label = "Precipitation")
    plt.ylabel("Rainfall Amount [In]", fontsize = 10)

plt.axhline(y=0, linestyle ="--", color="#0CE60A",linewidth=2) 
#positive = internal has higher values
#negative = external has higher values
tes = [] #calculates when the storm's center passes 35 N/S
te = []
for na in eqblahead:
    if np.isnan(na):
        te.append(na)
    else:
        tes.append(na)
if len(te) != 0:
    leng = len(tes)
    plt.axvline(x=x[leng-1]+timeahead, linestyle ="--", color="k", label = "Storm Passes 35{} N/S, Persists {} More Hours".format(chr(176), 6*len(te)))
breaks = 0 #so the label doesn't repeat
for lin in corr: #so I can show what times a storm box is not full size

    if (type(box_size[lin]) == int) & (breaks != 0): #so the label doesn't repeat
        
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B")
    if (type(box_size[lin]) == int) & (breaks == 0):
        plt.axvline(x=x[lin], linestyle =(0, (3, 1, 1, 1, 1, 1)), color="#838B8B", label = "Box is Not Full Size")
        breaks = breaks + 1
        
breaker = 0 #so the label doesn't repeat
for lan in corr: #so I can show what time values have a storm on land
    if long_array[lan] >= 180: #to correct the landmask
        long_array[lan] = long_array[lan]-360
    if globe.is_land(lati_array[lan], long_array[lan]) == True:
        if breaker != 0: #so the label doesn't repeat
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5)
        if breaker == 0:
            plt.axvline(x=x[lan], linestyle =(0, (3, 10, 1, 10, 1, 10)), color="k", alpha = .5, label = "Center Over land")
            breaker = breaker + 1
plt.axvline(x=x[pslmax], linewidth=2,label="Minimal Pressure: {:.4f}".format(pslmaxval))   
plt.grid()
plt.legend()
plt.xlabel("Time Step in Storm's Life", fontsize = 10)


plt.title('$\u00B1${} - Environment {} Time Steps Ago'.format(internal,timeahead),fontsize=12);
plt.suptitle('Tropical Cyclone # {:d} in year {:d}: {} Timesteps'.format(tc_number,year,type_of_plot),fontsize=18);
plt.tight_layout()

# Takes the Internal Minus the External and Internal Minus Ahead For Every Tropical Cyclone that Landfalls and is within the Tropics

In [ ]:
off_check = offset[0]-internal
off_range = (2*offset[0])-.25
for v in landfall: #taking every storm that landfalls and have a complete box in the tropics
    print(v)
    eqblint = [] #empty array for EQBL (internal)
    eqslftint= [] #empty array for EQSLFT (internal)
    equlftint = [] #empty array for EQULFT (internal)
    subsint = [] #empty array for subsaturation (internal)
    capeint = [] #empty array for CAPE (internal)
    buoyint = [] #empty array for buoyancy (internal)
    preciint = [] #empty array for precipitation (internal)
    
    
    
    eqblext = [] #empty array for EQBL (external)
    eqslftext= [] #empty array for EQSLFT (external)
    equlftext = [] #empty array for EQULFT (external)
    subsext = [] #empty array for subsaturation (external)
    capeext = [] #empty array for CAPE (external)
    buoyext = [] #empty array for buoyancy (external)
    preciext = [] #empty array for precipitation (external)
    
    tc_number = v
    lat_array=tcs.tc_tracks[tc_number]['lat'] #take latitude and longitude array
    lon_array=tcs.tc_tracks[tc_number]['lon'] 
    psl=tcs.tc_tracks[tc_number]['psl'] #takes the array of pressure at each storm's timestep
    maxwind=tcs.tc_tracks[tc_number]['maxwind'] #takes the array of max wind at each storm's timestep
    g = np.int_(np.linspace(0,tcs.ds_comp[tc_number].precipitation.time.size-1)) #so I get the exact number of frames
    newset=max(g)-timeahead
    
    z = np.arange(0,newset,1)
    eqblahead = []  #empty array for EQBL (ahead)
    eqslftahead= []  #empty array for EQSLFT (ahead)
    equlftahead = []  #empty array for EQULFT (ahead)
    subsahead = []  #empty array for subsaturation (ahead)
    capeahead = []  #empty array for CAPE (ahead)
    buoyahead = [] #empty array for buoyancy (ahead)
    preciahead = []  #empty array for precipitation (ahead)

    x = 0
    y = 0

    for t in range(max(g)+1):

        eqbl= tcs.ds_comp[tc_number].EQBL.isel(time=t) #taking the values for equivalent pot. temp. in the ABL
        eqblada=eqbl.stack(x=['lat','lon'])
        eqbla=eqblada[eqblada.notnull()] #so I can manipulate the values for this factor and take the inside and outside
    
        eqslft= tcs.ds_comp[tc_number].EQSLFT.isel(time=t) #taking the values for equivalent sat. pot. temp. in the LFT
        eqslftada=eqslft.stack(x=['lat','lon'])
        eqslfta=eqslftada[eqslftada.notnull()]
    
        equlft= tcs.ds_comp[tc_number].EQULFT.isel(time=t) #taking the values for equivalent unsat. pot. temp. in the LFT
        equlftada=equlft.stack(x=['lat','lon'])
        equlfta=equlftada[equlftada.notnull()]
        
        subs= tcs.ds_comp[tc_number].Subsaturation.isel(time=t) #taking the values for subsaturation in the LFT
        subsada_stacked=subs.stack(x=['lat','lon'])
        subsa=subsada_stacked[subsada_stacked.notnull()]
        
        cape= tcs.ds_comp[tc_number].CAPE.isel(time=t) #taking the values for CAPE in the LFT
        capada_stacked=cape.stack(x=['lat','lon'])
        capea=capada_stacked[capada_stacked.notnull()]
    
        buoy= tcs.ds_comp[tc_number].Buoyancy.isel(time=t) #taking the buoyancy for buoyancy in the LFT
        buoyada=buoy.stack(x=['lat','lon'])
        buoya=buoyada[buoyada.notnull()]
        
        preci= tcs.ds_comp[tc_number].precipitation.isel(time=t) #taking the values for precipitation
        preciada=preci.stack(x=['lat','lon'])
        precia=preciada[preciada.notnull()]
        #CALCULATE VALUES FOR EXTERNAL BOX
        max_long=lat_array[t]+outter
        min_long=lat_array[t]-outter
        if len(eqbla.lon.values) == 0:
            continue
        elif (max_long <= 34.75) & (min_long >=-34.75):
            r=max(eqbla.lon.values)
            o=min(eqbla.lon.values)
            pp=o+off_check
            qq=r-off_check

            eqblaorder=eqbla.sortby(eqbla.lat)
            rr=max(eqblaorder.lat.values)
            oo=min(eqblaorder.lat.values)
            if rr-oo == off_range: #for the box when full sized
                p=oo+off_check
                q=rr-off_check
                eqbl_check = eqbla.where((eqbla.lon<pp) | (eqbla.lon>qq) | (eqbla.lat<p) | (eqbla.lat>q))
                eqslft_check = eqslfta.where((eqslfta.lon<pp) | (eqslfta.lon>qq) | (eqslfta.lat<p) | (eqslfta.lat>q))
                equlft_check = equlfta.where((equlfta.lon<pp) | (equlfta.lon>qq) | (equlfta.lat<p) | (equlfta.lat>q))

                subs_check = subsa.where((subsa.lon<pp) | (subsa.lon>qq) | (subsa.lat<p) | (subsa.lat>q))
                cape_check = capea.where((capea.lon<pp) | (capea.lon>qq) | (capea.lat<p) | (capea.lat>q))

                buoy_check = buoya.where((buoya.lon<pp) | (buoya.lon>qq) | (buoya.lat<p) | (buoya.lat>q))
                preci_check = precia.where((precia.lon<pp) | (precia.lon>qq) | (precia.lat<p) | (precia.lat>q))

                x = x + 1

            else:
                continue
        #APPEND TO EXTERNAL LIST
            eqblext.append(np.nanmean(eqbl_check))
            eqslftext.append(np.nanmean(eqslft_check))
            equlftext.append(np.nanmean(equlft_check))
            subsext.append(np.nanmean(subs_check))
            capeext.append(np.nanmean(cape_check))
            buoyext.append(np.nanmean(buoy_check))
            preciext.append(np.nanmean(preci_check))
            #CALCULATE BOXES FOR INTERNAL VALUES
            max_long=lat_array[t]+outter
            min_long=lat_array[t]-outter
            if (max_long <= 34.75) & (min_long >=-34.75):
                r=lon_array[t]+internal
                o=lon_array[t]-internal
                pp=o
                qq=r
                rr=lat_array[t]+internal
                oo=lat_array[t]-internal
                p=oo
                q=rr
                eqbl_check = eqbla.where((eqbla.lon>=pp) & (eqbla.lon<=qq) & (eqbla.lat>=p) & (eqbla.lat<=q))
                eqslft_check = eqslfta.where((eqslfta.lon>=pp) & (eqslfta.lon<=qq) & (eqslfta.lat>=p) & (eqslfta.lat<=q))
                equlft_check = equlfta.where((equlfta.lon>=pp) & (equlfta.lon<=qq) & (equlfta.lat>=p) & (equlfta.lat<=q))

                subs_check = subsa.where((subsa.lon>=pp) & (subsa.lon<=qq) & (subsa.lat>=p) & (subsa.lat<=q))
                cape_check = capea.where((capea.lon>=pp) & (capea.lon<=qq) & (capea.lat>=p) & (capea.lat<=q))

                buoy_check = buoya.where((buoya.lon>=pp) & (buoya.lon<=qq) & (buoya.lat>=p) & (buoya.lat<=q))
                preci_check = precia.where((precia.lon>=pp) & (precia.lon<=qq) & (precia.lat>=p) & (precia.lat<=q))
            else:
                continue
            #CALCULATE BOXES FOR AHEAD VALUES
            if t <= newset:
                ma_long=lat_array[t+timeahead]+outter
                
                mi_long=lat_array[t+timeahead]-outter
                
                if (ma_long <= 34.75) & (mi_long >=-34.75):
                    r=lon_array[t+timeahead]+internal
                    o=lon_array[t+timeahead]-internal
                    pp=o
                    qq=r
                    rr=lat_array[t+timeahead]+internal
                    oo=lat_array[t+timeahead]-internal
                    p=oo
                    q=rr
                    eqbl_ahe = eqbla.where((eqbla.lon>=pp) & (eqbla.lon<=qq) & (eqbla.lat>=p) & (eqbla.lat<=q))
                    eqslft_ahe = eqslfta.where((eqslfta.lon>=pp) & (eqslfta.lon<=qq) & (eqslfta.lat>=p) & (eqslfta.lat<=q))
                    equlft_ahe = equlfta.where((equlfta.lon>=pp) & (equlfta.lon<=qq) & (equlfta.lat>=p) & (equlfta.lat<=q))

                    subs_ahe = subsa.where((subsa.lon>=pp) & (subsa.lon<=qq) & (subsa.lat>=p) & (subsa.lat<=q))
                    cape_ahe = capea.where((capea.lon>=pp) & (capea.lon<=qq) & (capea.lat>=p) & (capea.lat<=q))

                    buoy_ahe = buoya.where((buoya.lon>=pp) & (buoya.lon<=qq) & (buoya.lat>=p) & (buoya.lat<=q))
                    preci_ahe = precia.where((precia.lon>=pp) & (precia.lon<=qq) & (precia.lat>=p) & (precia.lat<=q))                
                    y=y+1

            #APPEND INTERNAL VALUES
            eqblint.append(np.nanmean(eqbl_check)) #appending to the external list
            eqslftint.append(np.nanmean(eqslft_check))
            equlftint.append(np.nanmean(equlft_check))
            subsint.append(np.nanmean(subs_check))
            capeint.append(np.nanmean(cape_check))
            buoyint.append(np.nanmean(buoy_check))
            preciint.append(np.nanmean(preci_check))
            #APPEND AHEAD VALUES
            eqblahead.append(np.nanmean(eqbl_ahe))
            eqslftahead.append(np.nanmean(eqslft_ahe))
            equlftahead.append(np.nanmean(equlft_ahe))
            subsahead.append(np.nanmean(subs_ahe))
            capeahead.append(np.nanmean(cape_ahe))
            buoyahead.append(np.nanmean(buoy_ahe))
            preciahead.append(np.nanmean(preci_ahe))
   
    #AVERAGE THE DIFFERENCE BETWEEN THE INTERNAL VALUES AND THE EXTERNAL VALUES
    EQBL_weight_unweight = [0] * len(range(x)) #creating empty array to take the difference between internal and external
    for i in range(x):
        EQBL_weight_unweight[i]= (eqblint[i] - eqblext[i]) #taking the difference between internal and external average
  
    
    EQSLFT_weight_unweight  = [0] * len(range(x))
    for i in range(x):
        EQSLFT_weight_unweight[i]= (eqslftint[i] - eqslftext[i])
  

    EQULFT_weight_unweight  = [0] * len(range(x)) 
    for i in range(x):
        EQULFT_weight_unweight[i]= (equlftint[i] - equlftext[i])
        
    Subsat_weight_unweight = [0] * len(range(x)) 
    for i in range(x):
        Subsat_weight_unweight[i]= (subsint[i] - subsext[i])
    
    CAPE_weight_unweight = [0] * len(range(x)) 
    for i in range(x):
        CAPE_weight_unweight[i]= (capeint[i] - capeext[i])
    
    Buoy_weight_unweight = [0] * len(range(x)) 
    for i in range(x):
        Buoy_weight_unweight[i]= (buoyint[i] - buoyext[i]) 
        
    Precip_weight_unweight = [0] * len(range(x)) 
    for i in range(x):
        Precip_weight_unweight[i]= (preciint[i] - preciext[i])
        
    #AVERAGE THE DIFFERENCE BETWEEN THE INTERNAL VALUES AND THE AHEAD VALUES
    EQBL_ahead = [0] * len(range(y)) #creating empty array to take the difference between internal and ahead
    for i in range(y):
        EQBL_ahead[i]= (eqblint[i+2] - eqblahead[i]) #taking the difference between internal and ahead average
    
    EQSLFT_ahead  = [0] * len(range(y))
    for i in range(y):
        EQSLFT_ahead[i]= (eqslftint[i+2] - eqslftahead[i])
  

    EQULFT_ahead  = [0] * len(range(y)) 
    for i in range(y):
        EQULFT_ahead[i]= (equlftint[i+2] - equlftahead[i])
        
    Subsat_ahead = [0] * len(range(y)) 
    for i in range(y):
        Subsat_ahead[i]= (subsint[i+2] - subsahead[i])
    
    CAPE_ahead = [0] * len(range(y)) 
    for i in range(y):
        CAPE_ahead[i]= (capeint[i+2] - capeahead[i])
    
    Buoy_ahead = [0] * len(range(y)) 
    for i in range(y):
        Buoy_ahead[i]= (buoyint[i+2] - buoyahead[i]) 
        
    Precip_ahead = [0] * len(range(y)) 
    for i in range(y):
        Precip_ahead[i]= (preciint[i+2] - preciahead[i])
    
    f=open('Buoy_Expanded_Landfall_New_Fix.txt','a') #so I can create a text file and create graphs of values
    for i in range(len(EQBL_weight_unweight)): #since the length of each list is the same
        
        lines = ['{}'.format(tc_number), ' ', "{}".format(psl[i]), ' ', "{}".format(maxwind[i]), ' ', "{}".format(EQBL_weight_unweight[i]), ' ', "{}".format(EQSLFT_weight_unweight[i]), ' ', "{}".format(EQULFT_weight_unweight[i]), ' ', "{}".format(Subsat_weight_unweight[i]),' ', "{}".format(CAPE_weight_unweight[i]), ' ', "{}".format(Buoy_weight_unweight[i]), ' ', "{}".format(Precip_weight_unweight[i])]
        f.writelines(lines) #writing in the lines list above
        f.write('\n')
    f.close() #close file
    
    f=open('Buoy_Expanded_Landfall_Ahead_Fix.txt','a') #so I can create a text file and create graphs of values
    for i in range(len(EQBL_ahead)): #since the length of each list is the same
        
        lines = ['{}'.format(tc_number), ' ', "{}".format(psl[i]), ' ', "{}".format(maxwind[i]), ' ', "{}".format(EQBL_ahead[i]), ' ', "{}".format(EQSLFT_ahead[i]), ' ', "{}".format(EQULFT_ahead[i]), ' ', "{}".format(Subsat_ahead[i]),' ', "{}".format(CAPE_ahead[i]), ' ', "{}".format(Buoy_ahead[i]), ' ', "{}".format(Precip_ahead[i])]
        f.writelines(lines) #writing in the lines list above
        f.write('\n')
    f.close() #close file
    
    f=open('Buoy_Expanded_Landfall_Internal_Fix.txt','a') #so I can create a text file and create graphs of values
    for i in range(len(EQBL_weight_unweight)): #since the length of each list is the same
        
        lines = ['{}'.format(tc_number), ' ', "{}".format(psl[i]), ' ', "{}".format(maxwind[i]), ' ', "{}".format(eqblint[i]), ' ', "{}".format(eqslftint[i]), ' ', "{}".format(equlftint[i]), ' ', "{}".format(subsint[i]),' ', "{}".format(capeint[i]), ' ', "{}".format(buoyint[i]), ' ', "{}".format(preciint[i])]
        f.writelines(lines) #writing in the lines list above
        f.write('\n')
    f.close() #close file
    
    f=open('Buoy_Expanded_Landfall_External_Fix.txt','a') #so I can create a text file and create graphs of values
    for i in range(len(EQBL_weight_unweight)): #since the length of each list is the same
        
        lines = ['{}'.format(tc_number), ' ', "{}".format(psl[i]), ' ', "{}".format(maxwind[i]), ' ', "{}".format(eqblext[i]), ' ', "{}".format(eqslftext[i]), ' ', "{}".format(equlftext[i]), ' ', "{}".format(subsext[i]),' ', "{}".format(capeext[i]), ' ', "{}".format(buoyext[i]), ' ', "{}".format(preciext[i])]
        f.writelines(lines) #writing in the lines list above
        f.write('\n')
    f.close() #close file
    
    f=open('Buoy_Expanded_Landfall_Proj_Fix.txt','a') #so I can create a text file and create graphs of values
    for i in range(len(EQBL_ahead)): #since the length of each list is the same
        
        lines = ['{}'.format(tc_number), ' ', "{}".format(psl[i]), ' ', "{}".format(maxwind[i]), ' ', "{}".format(eqblahead[i]), ' ', "{}".format(eqslftahead[i]), ' ', "{}".format(equlftahead[i]), ' ', "{}".format(subsahead[i]),' ', "{}".format(capeahead[i]), ' ', "{}".format(buoyahead[i]), ' ', "{}".format(preciahead[i])]
        f.writelines(lines) #writing in the lines list above
        f.write('\n')
    f.close() #close file
print("Done")


# Takes the Internal Minus the External and Internal Minus Ahead For Every Tropical Cyclone that Doesn't Landfall and is within the Tropics

In [ ]:
off_check = offset[0]-internal
off_range = (2*offset[0])-.25
for v in non_landfall: #taking every storm that landfalls and have a complete box in the tropics
    print(v)
    eqblint = [] #empty array for EQBL (internal)
    eqslftint= [] #empty array for EQSLFT (internal)
    equlftint = [] #empty array for EQULFT (internal)
    subsint = [] #empty array for subsaturation (internal)
    capeint = [] #empty array for CAPE (internal)
    buoyint = [] #empty array for buoyancy (internal)
    preciint = [] #empty array for precipitation (internal)
    
    
    
    eqblext = [] #empty array for EQBL (external)
    eqslftext= [] #empty array for EQSLFT (external)
    equlftext = [] #empty array for EQULFT (external)
    subsext = [] #empty array for subsaturation (external)
    capeext = [] #empty array for CAPE (external)
    buoyext = [] #empty array for buoyancy (external)
    preciext = [] #empty array for precipitation (external)
    
    tc_number = v
    lat_array=tcs.tc_tracks[tc_number]['lat'] #take latitude and longitude array
    lon_array=tcs.tc_tracks[tc_number]['lon'] 
    psl=tcs.tc_tracks[tc_number]['psl'] #takes the array of pressure at each storm's timestep
    maxwind=tcs.tc_tracks[tc_number]['maxwind'] #takes the array of max wind at each storm's timestep
    g = np.int_(np.linspace(0,tcs.ds_comp[tc_number].precipitation.time.size-1)) #so I get the exact number of frames
    newset=max(g)-timeahead
    
    z = np.arange(0,newset,1)
    eqblahead = []  #empty array for EQBL (ahead)
    eqslftahead= []  #empty array for EQSLFT (ahead)
    equlftahead = []  #empty array for EQULFT (ahead)
    subsahead = []  #empty array for subsaturation (ahead)
    capeahead = []  #empty array for CAPE (ahead)
    buoyahead = [] #empty array for buoyancy (ahead)
    preciahead = []  #empty array for precipitation (ahead)

    x = 0
    y = 0

    for t in range(max(g)+1):

        eqbl= tcs.ds_comp[tc_number].EQBL.isel(time=t) #taking the values for equivalent pot. temp. in the ABL
        eqblada=eqbl.stack(x=['lat','lon'])
        eqbla=eqblada[eqblada.notnull()] #so I can manipulate the values for this factor and take the inside and outside
    
        eqslft= tcs.ds_comp[tc_number].EQSLFT.isel(time=t) #taking the values for equivalent sat. pot. temp. in the LFT
        eqslftada=eqslft.stack(x=['lat','lon'])
        eqslfta=eqslftada[eqslftada.notnull()]
    
        equlft= tcs.ds_comp[tc_number].EQULFT.isel(time=t) #taking the values for equivalent unsat. pot. temp. in the LFT
        equlftada=equlft.stack(x=['lat','lon'])
        equlfta=equlftada[equlftada.notnull()]
        
        subs= tcs.ds_comp[tc_number].Subsaturation.isel(time=t) #taking the values for subsaturation in the LFT
        subsada_stacked=subs.stack(x=['lat','lon'])
        subsa=subsada_stacked[subsada_stacked.notnull()]
        
        cape= tcs.ds_comp[tc_number].CAPE.isel(time=t) #taking the values for CAPE in the LFT
        capada_stacked=cape.stack(x=['lat','lon'])
        capea=capada_stacked[capada_stacked.notnull()]
    
        buoy= tcs.ds_comp[tc_number].Buoyancy.isel(time=t) #taking the buoyancy for buoyancy in the LFT
        buoyada=buoy.stack(x=['lat','lon'])
        buoya=buoyada[buoyada.notnull()]
        
        preci= tcs.ds_comp[tc_number].precipitation.isel(time=t) #taking the values for precipitation
        preciada=preci.stack(x=['lat','lon'])
        precia=preciada[preciada.notnull()]
        #CALCULATE VALUES FOR EXTERNAL BOX
        max_long=lat_array[t]+outter
        min_long=lat_array[t]-outter
        if len(eqbla.lon.values) == 0:
            continue
        
        elif (max_long <= 34.75) & (min_long >=-34.75):
            r=max(eqbla.lon.values)
            o=min(eqbla.lon.values)
            pp=o+off_check
            qq=r-off_check

            eqblaorder=eqbla.sortby(eqbla.lat)
            rr=max(eqblaorder.lat.values)
            oo=min(eqblaorder.lat.values)
            if rr-oo == off_range: #for the box when full sized
                p=oo+off_check
                q=rr-off_check
                eqbl_check = eqbla.where((eqbla.lon<pp) | (eqbla.lon>qq) | (eqbla.lat<p) | (eqbla.lat>q))
                eqslft_check = eqslfta.where((eqslfta.lon<pp) | (eqslfta.lon>qq) | (eqslfta.lat<p) | (eqslfta.lat>q))
                equlft_check = equlfta.where((equlfta.lon<pp) | (equlfta.lon>qq) | (equlfta.lat<p) | (equlfta.lat>q))

                subs_check = subsa.where((subsa.lon<pp) | (subsa.lon>qq) | (subsa.lat<p) | (subsa.lat>q))
                cape_check = capea.where((capea.lon<pp) | (capea.lon>qq) | (capea.lat<p) | (capea.lat>q))

                buoy_check = buoya.where((buoya.lon<pp) | (buoya.lon>qq) | (buoya.lat<p) | (buoya.lat>q))
                preci_check = precia.where((precia.lon<pp) | (precia.lon>qq) | (precia.lat<p) | (precia.lat>q))

                x = x + 1

            else:
                continue
        #APPEND TO EXTERNAL LIST
            eqblext.append(np.nanmean(eqbl_check))
            eqslftext.append(np.nanmean(eqslft_check))
            equlftext.append(np.nanmean(equlft_check))
            subsext.append(np.nanmean(subs_check))
            capeext.append(np.nanmean(cape_check))
            buoyext.append(np.nanmean(buoy_check))
            preciext.append(np.nanmean(preci_check))
            #CALCULATE BOXES FOR INTERNAL VALUES
            max_long=lat_array[t]+outter
            min_long=lat_array[t]-outter
            if (max_long <= 34.75) & (min_long >=-34.75):
                r=lon_array[t]+internal
                o=lon_array[t]-internal
                pp=o
                qq=r
                rr=lat_array[t]+internal
                oo=lat_array[t]-internal
                p=oo
                q=rr
                eqbl_check = eqbla.where((eqbla.lon>=pp) & (eqbla.lon<=qq) & (eqbla.lat>=p) & (eqbla.lat<=q))
                eqslft_check = eqslfta.where((eqslfta.lon>=pp) & (eqslfta.lon<=qq) & (eqslfta.lat>=p) & (eqslfta.lat<=q))
                equlft_check = equlfta.where((equlfta.lon>=pp) & (equlfta.lon<=qq) & (equlfta.lat>=p) & (equlfta.lat<=q))

                subs_check = subsa.where((subsa.lon>=pp) & (subsa.lon<=qq) & (subsa.lat>=p) & (subsa.lat<=q))
                cape_check = capea.where((capea.lon>=pp) & (capea.lon<=qq) & (capea.lat>=p) & (capea.lat<=q))

                buoy_check = buoya.where((buoya.lon>=pp) & (buoya.lon<=qq) & (buoya.lat>=p) & (buoya.lat<=q))
                preci_check = precia.where((precia.lon>=pp) & (precia.lon<=qq) & (precia.lat>=p) & (precia.lat<=q))
            else:
                continue
            #CALCULATE BOXES FOR AHEAD VALUES
            if t <= newset:
                ma_long=lat_array[t+timeahead]+outter
                
                mi_long=lat_array[t+timeahead]-outter
                
                if (ma_long <= 34.75) & (mi_long >=-34.75):
                    r=lon_array[t+timeahead]+internal
                    o=lon_array[t+timeahead]-internal
                    pp=o
                    qq=r
                    rr=lat_array[t+timeahead]+internal
                    oo=lat_array[t+timeahead]-internal
                    p=oo
                    q=rr
                    eqbl_ahe = eqbla.where((eqbla.lon>=pp) & (eqbla.lon<=qq) & (eqbla.lat>=p) & (eqbla.lat<=q))
                    eqslft_ahe = eqslfta.where((eqslfta.lon>=pp) & (eqslfta.lon<=qq) & (eqslfta.lat>=p) & (eqslfta.lat<=q))
                    equlft_ahe = equlfta.where((equlfta.lon>=pp) & (equlfta.lon<=qq) & (equlfta.lat>=p) & (equlfta.lat<=q))

                    subs_ahe = subsa.where((subsa.lon>=pp) & (subsa.lon<=qq) & (subsa.lat>=p) & (subsa.lat<=q))
                    cape_ahe = capea.where((capea.lon>=pp) & (capea.lon<=qq) & (capea.lat>=p) & (capea.lat<=q))

                    buoy_ahe = buoya.where((buoya.lon>=pp) & (buoya.lon<=qq) & (buoya.lat>=p) & (buoya.lat<=q))
                    preci_ahe = precia.where((precia.lon>=pp) & (precia.lon<=qq) & (precia.lat>=p) & (precia.lat<=q))                
                    y=y+1

            #APPEND INTERNAL VALUES
            eqblint.append(np.nanmean(eqbl_check)) #appending to the external list
            eqslftint.append(np.nanmean(eqslft_check))
            equlftint.append(np.nanmean(equlft_check))
            subsint.append(np.nanmean(subs_check))
            capeint.append(np.nanmean(cape_check))
            buoyint.append(np.nanmean(buoy_check))
            preciint.append(np.nanmean(preci_check))
            #APPEND AHEAD VALUES
            eqblahead.append(np.nanmean(eqbl_ahe))
            eqslftahead.append(np.nanmean(eqslft_ahe))
            equlftahead.append(np.nanmean(equlft_ahe))
            subsahead.append(np.nanmean(subs_ahe))
            capeahead.append(np.nanmean(cape_ahe))
            buoyahead.append(np.nanmean(buoy_ahe))
            preciahead.append(np.nanmean(preci_ahe))
   
    #AVERAGE THE DIFFERENCE BETWEEN THE INTERNAL VALUES AND THE EXTERNAL VALUES
    EQBL_weight_unweight = [0] * len(range(x)) #creating empty array to take the difference between internal and external
    for i in range(x):
        EQBL_weight_unweight[i]= (eqblint[i] - eqblext[i]) #taking the difference between internal and external average
  
    
    EQSLFT_weight_unweight  = [0] * len(range(x))
    for i in range(x):
        EQSLFT_weight_unweight[i]= (eqslftint[i] - eqslftext[i])
  

    EQULFT_weight_unweight  = [0] * len(range(x)) 
    for i in range(x):
        EQULFT_weight_unweight[i]= (equlftint[i] - equlftext[i])
        
    Subsat_weight_unweight = [0] * len(range(x)) 
    for i in range(x):
        Subsat_weight_unweight[i]= (subsint[i] - subsext[i])
    
    CAPE_weight_unweight = [0] * len(range(x)) 
    for i in range(x):
        CAPE_weight_unweight[i]= (capeint[i] - capeext[i])
    
    Buoy_weight_unweight = [0] * len(range(x)) 
    for i in range(x):
        Buoy_weight_unweight[i]= (buoyint[i] - buoyext[i]) 
        
    Precip_weight_unweight = [0] * len(range(x)) 
    for i in range(x):
        Precip_weight_unweight[i]= (preciint[i] - preciext[i])
        
    #AVERAGE THE DIFFERENCE BETWEEN THE INTERNAL VALUES AND THE AHEAD VALUES

    EQBL_ahead = [0] * len(range(y)) #creating empty array to take the difference between internal and ahead
    for i in range(y):
        EQBL_ahead[i]= (eqblint[i+2] - eqblahead[i]) #taking the difference between internal and ahead average
    
    EQSLFT_ahead  = [0] * len(range(y))
    for i in range(y):
        EQSLFT_ahead[i]= (eqslftint[i+2] - eqslftahead[i])
  

    EQULFT_ahead  = [0] * len(range(y)) 
    for i in range(y):
        EQULFT_ahead[i]= (equlftint[i+2] - equlftahead[i])
        
    Subsat_ahead = [0] * len(range(y)) 
    for i in range(y):
        Subsat_ahead[i]= (subsint[i+2] - subsahead[i])
    
    CAPE_ahead = [0] * len(range(y)) 
    for i in range(y):
        CAPE_ahead[i]= (capeint[i+2] - capeahead[i])
    
    Buoy_ahead = [0] * len(range(y)) 
    for i in range(y):
        Buoy_ahead[i]= (buoyint[i+2] - buoyahead[i]) 
        
    Precip_ahead = [0] * len(range(y)) 
    for i in range(y):
        Precip_ahead[i]= (preciint[i+2] - preciahead[i])
    
    f=open('Buoy_Expanded_Non_Landfall_New_Fix.txt','a') #so I can create a text file and create graphs of values
    for i in range(len(EQBL_weight_unweight)): #since the length of each list is the same
        
        lines = ['{}'.format(tc_number), ' ', "{}".format(psl[i]), ' ', "{}".format(maxwind[i]), ' ', "{}".format(EQBL_weight_unweight[i]), ' ', "{}".format(EQSLFT_weight_unweight[i]), ' ', "{}".format(EQULFT_weight_unweight[i]), ' ', "{}".format(Subsat_weight_unweight[i]),' ', "{}".format(CAPE_weight_unweight[i]), ' ', "{}".format(Buoy_weight_unweight[i]), ' ', "{}".format(Precip_weight_unweight[i])]
        f.writelines(lines) #writing in the lines list above
        f.write('\n')
    f.close() #close file
    
    f=open('Buoy_Expanded_Non_Landfall_Ahead_Fix.txt','a') #so I can create a text file and create graphs of values
    for i in range(len(EQBL_ahead)): #since the length of each list is the same
        
        lines = ['{}'.format(tc_number), ' ', "{}".format(psl[i]), ' ', "{}".format(maxwind[i]), ' ', "{}".format(EQBL_ahead[i]), ' ', "{}".format(EQSLFT_ahead[i]), ' ', "{}".format(EQULFT_ahead[i]), ' ', "{}".format(Subsat_ahead[i]),' ', "{}".format(CAPE_ahead[i]), ' ', "{}".format(Buoy_ahead[i]), ' ', "{}".format(Precip_ahead[i])]
        f.writelines(lines) #writing in the lines list above
        f.write('\n')
    f.close() #close file
    
    f=open('Buoy_Expanded_Non_Landfall_Internal_Fix.txt','a') #so I can create a text file and create graphs of values
    for i in range(len(EQBL_weight_unweight)): #since the length of each list is the same
        
        lines = ['{}'.format(tc_number), ' ', "{}".format(psl[i]), ' ', "{}".format(maxwind[i]), ' ', "{}".format(eqblint[i]), ' ', "{}".format(eqslftint[i]), ' ', "{}".format(equlftint[i]), ' ', "{}".format(subsint[i]),' ', "{}".format(capeint[i]), ' ', "{}".format(buoyint[i]), ' ', "{}".format(preciint[i])]
        f.writelines(lines) #writing in the lines list above
        f.write('\n')
    f.close() #close file
    
    f=open('Buoy_Expanded_Non_Landfall_External_Fix.txt','a') #so I can create a text file and create graphs of values
    for i in range(len(EQBL_weight_unweight)): #since the length of each list is the same
        
        lines = ['{}'.format(tc_number), ' ', "{}".format(psl[i]), ' ', "{}".format(maxwind[i]), ' ', "{}".format(eqblext[i]), ' ', "{}".format(eqslftext[i]), ' ', "{}".format(equlftext[i]), ' ', "{}".format(subsext[i]),' ', "{}".format(capeext[i]), ' ', "{}".format(buoyext[i]), ' ', "{}".format(preciext[i])]
        f.writelines(lines) #writing in the lines list above
        f.write('\n')
    f.close() #close file
    
    f=open('Buoy_Expanded_Non_Landfall_Proj_Fix.txt','a') #so I can create a text file and create graphs of values
    for i in range(len(EQBL_ahead)): #since the length of each list is the same
        
        lines = ['{}'.format(tc_number), ' ', "{}".format(psl[i]), ' ', "{}".format(maxwind[i]), ' ', "{}".format(eqblahead[i]), ' ', "{}".format(eqslftahead[i]), ' ', "{}".format(equlftahead[i]), ' ', "{}".format(subsahead[i]),' ', "{}".format(capeahead[i]), ' ', "{}".format(buoyahead[i]), ' ', "{}".format(preciahead[i])]
        f.writelines(lines) #writing in the lines list above
        f.write('\n')
    f.close() #close file
print("Done")

